In [ ]:
from gensim.models import Word2Vec

In [ ]:
!git clone https://github.com/stanfordnlp/GloVe.git

def word_analogy_evaluation(vectors_file): 
  import torch
  # in this function the vector file should be devrivative from the start to the return
  # so we need not to make any changes on this file to make the loss to be derivative
  # however, there are few changes here: like 
  # wait, firstly, we need input a torch tensor but not a np array (for the vectors file)
  # secondly, we need no use of like these sentences:
  # assignment      W[vocab[word], :] = torch.from_numpy(v)
  # slice     pred_vec = (W[ind2[subset], :] - W[ind1[subset], :]+  W[ind3[subset], :])
  # assginemt      expected_matrix[ind4[subset[setid]]][setid]=1 #第i个列最大的数值为1，为真值，其行坐标为ind14[i]
  # no change for the leaf node
  # tensor ---> req-grad =False--> f(tensor)--->req-grad =True->return value
  # for f, only computation formulas include tensor, no value changes for this one

  vectors = vectors_file

  vocab_words=list(vectors.keys())
  vocab_size = len(vocab_words)

  # create word->index and index->word converter
  vocab = {w: idx for idx, w in enumerate(vocab_words)}
  ivocab = {idx: w for idx, w in enumerate(vocab_words)}

  # create the embedding matrix of shape (vocab_size, dim)
  vector_dim = len(vectors[ivocab[0]])
  #W = np.zeros((vocab_size, vector_dim))
  #W = torch.from_numpy(W)
  W = torch.zeros(vocab_size, vector_dim).float()
  #print(W.requires_grad)
  #print(W.dtype)
  #W.requires_grad_(True)
  for word, v in vectors.items():
      if word == '<unk>':
          continue
      W[vocab[word], :] = torch.from_numpy(v)

  # normalize each word vector to unit length
  # Vectors are usually normalized to unit length before they are used for similarity calculation, making cosine similarity and dot-product equivalent.
  W_norm = torch.zeros(vocab_size, vector_dim).float()
  d = (torch.sum(W ** 2, 1) ** (0.5))
  W_norm = (W.T / d).T
    
  def evaluate_vectors(W,vocab, prefix='./eval/question-data/'):
        """Evaluate the trained word vectors on a variety of tasks"""

        filenames = [
            'capital-common-countries.txt', 'capital-world.txt', 'currency.txt',
            'city-in-state.txt', 'family.txt', 'gram1-adjective-to-adverb.txt',
            'gram2-opposite.txt', 'gram3-comparative.txt', 'gram4-superlative.txt',
            'gram5-present-participle.txt', 'gram6-nationality-adjective.txt',
            'gram7-past-tense.txt', 'gram8-plural.txt', 'gram9-plural-verbs.txt',
            ]

        # to avoid memory overflow, could be increased/decreased
        # depending on system and vocab size
        split_size = 100

        correct_sem = 0; # count correct semantic questions
        correct_syn = 0; # count correct syntactic questions
        correct_tot = 0 # count correct questions
        count_sem = 0; # count all semantic questions
        count_syn = 0; # count all syntactic questions
        count_tot = 0 # count all questions
        full_count = 0 # count all questions, including those with unknown words

        for i in range(len(filenames)):
            with open('%s/%s' % (prefix, filenames[i]), 'r') as f:
                full_data = [line.rstrip().split(' ') for line in f]
                full_count += len(full_data)
                data = [x for x in full_data if all(word in vocab for word in x)]

            if len(data) == 0:
                #print("ERROR: no lines of vocab kept for %s !" % filenames[i])
                #print("Example missing line:", full_data[0])
                continue

            indices = np.array([[vocab[word] for word in row] for row in data])#Note: Great Problem
            indices = torch.LongTensor(indices)
            ind1, ind2, ind3, ind4 = indices.T
            ind1 = torch.LongTensor(ind1)
            ind2 = torch.LongTensor(ind2)
            ind3 = torch.LongTensor(ind3)
            ind4 = torch.LongTensor(ind4)

            predictions = np.zeros((len(indices),))
            predictions = torch.from_numpy(predictions)
            num_iter = int(np.ceil(len(indices) / float(split_size)))
            tot_loss1 = 0
            tot_loss2 = 0
            for j in range(num_iter): #每100个连续的词语做成一个batch进行训练
                subset = np.arange(j*split_size, min((j + 1)*split_size, len(ind1))) #subset用于对set中id->整个corpus词语id的重新定位
                subset = torch.LongTensor(subset)
                
                pred_vec = (W[ind2[subset], :] - W[ind1[subset], :]
                    +  W[ind3[subset], :])

                # normalization
                W_norm_pred = np.zeros(pred_vec.shape)
                W_norm_pred = torch.from_numpy(W_norm_pred)
                d_pred = (torch.sum(pred_vec ** 2, 1) ** (0.5))
                W_norm_pred = (pred_vec.T / d_pred).T

                dist = torch.mm(W, pred_vec.T)

                for k in range(len(subset)):
                    dist[ind1[subset[k]], k] = 0
                    dist[ind2[subset[k]], k] = 0
                    dist[ind3[subset[k]], k] = 0

                # predicted word index
                tmp = torch.argmax(dist,0).flatten()
                tmp = torch.FloatTensor(tmp.float())
                predictions[subset] = tmp.double()#torch.argmax(dist,0).flatten()
            
            dist.requires_grad_(True)
            
            ### construct expected matrix
            expected_matrix = 0.01*np.ones((W.shape[0],len(subset))) #所有词语数目(用于找到wordvec最相近词),此过程中的ind4数目
            expected_matrix = torch.from_numpy(expected_matrix)
            for setid in range(0,len(subset)): #ind4[i]代表对应的正确id
                expected_matrix[ind4[subset[setid]]][setid]=1 #第i个列最大的数值为1，为真值，其行坐标为ind14[i]
            
            expected_matrix.requires_grad_(True)
            
           ### calculate the angle between [0.01,,,,1,,,0.01]
            tot_loss2 += torch.sum(dist*expected_matrix) #similarity of prediction result matrix

            val = (ind4 == predictions) # correct predictions
            for sid in range(0,len(ind4)):
                if(ind4[sid]!=predictions[sid]):
                    tot_loss1 += torch.sum(W[ind4[sid]]*W[int(predictions[sid])])
            count_tot = count_tot + len(ind1)
            correct_tot = correct_tot + sum(val)
            if i < 5:
                count_sem = count_sem + len(ind1)
                correct_sem = correct_sem + sum(val)
            else:
                count_syn = count_syn + len(ind1)
                correct_syn = correct_syn + sum(val)
                
            tot_loss1.requires_grad_(True)
            tot_loss2.requires_grad_(True)
        
        return correct_sem, correct_syn, correct_tot, count_sem, count_syn, count_tot, full_count, tot_loss1, tot_loss2

  correct_sem, correct_syn, correct_tot, count_sem, count_syn, count_tot, full_count,tot_loss1, tot_loss2 \
  = evaluate_vectors(W,vocab, prefix='./GloVe/eval/question-data')
    
  semantic_acc = 100 * correct_sem / float(count_sem)
  syntactic_acc = 100 * correct_syn / float(count_syn)
  total_acc = 100 * correct_tot / float(count_tot)
  tot_loss1 =  100*tot_loss1 / float(vocab_size) #100--embedding vector length
  tot_loss2 =tot_loss2 / float(vocab_size) #average of similarity

  #print(total_acc)
  #print(tot_loss1)
  #print(tot_loss2)
  
  return semantic_acc, syntactic_acc, total_acc, tot_loss1, tot_loss2

Cloning into 'GloVe'...
remote: Enumerating objects: 606, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 606 (delta 5), reused 8 (delta 4), pack-reused 592
Receiving objects: 100% (606/606), 224.87 KiB | 1005.00 KiB/s, done.
Resolving deltas: 100% (343/343), done.


In [ ]:
def update_embedding(emb_table):
  # model for Text
  import torch
  import torch.nn as nn
  import torch.optim as optim
  from torch.autograd import Variable
  import torch.nn.functional as F

  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

  class Bi_LSTM_Attention(nn.Module):
      def __init__(self):
          super(Bi_LSTM_Attention, self).__init__()
          self.emb = nn.Embedding(emb_table.shape[0],emb_table.shape[1])
          self.emb.weight.data.copy_(torch.from_numpy(emb_table))
          self.emb.weight.requires_grad = True
          self.lstm = nn.LSTM(emb_table.shape[1], n_hidden, bidirectional=True)
          self.encoder_fc = nn.Linear(2*n_hidden, n_class)
          #self.activation = nn.ReLU()
      
      # https://colab.research.google.com/github/ngduyanhece/nlp-tutorial/blob/master/4-3.Bi-LSTM%28Attention%29/Bi_LSTM%28Attention%29_Torch.ipynb
      # output : [batch_size, n_step, n_hidden * num_directions(=2)], F matrix
      def attention_net(self, output, final_state):
          hidden = final_state.view(-1, n_hidden * 2, 1)   # hidden : [batch_size, n_hidden * num_directions(=2), 1(=n_layer)]
          attn_weights = torch.bmm(output, hidden).squeeze(2) # attn_weights : [batch_size, n_step]
          soft_attn_weights = F.softmax(attn_weights, 1).to(device)
          # [batch_size, n_hidden * num_directions(=2), n_step] * [batch_size, n_step, 1] = [batch_size, n_hidden * num_directions(=2), 1]
          context = torch.bmm(output.transpose(1, 2), soft_attn_weights.unsqueeze(2)).squeeze(2).to(device)
          #return context, soft_attn_weights # context : [batch_size, n_hidden * num_directions(=2)]
          return context, soft_attn_weights.data.numpy() # context : [batch_size, n_hidden * num_directions(=2)]

      def forward(self, X):
          x = self.emb(X) # input : [batch_size, len_seq, embedding_dim]
          # 32, 28, 100
          x = x.permute(1, 0, 2) # input : [len_seq, batch_size, embedding_dim]
          hidden_state = Variable(torch.zeros(1*2, x.shape[1], n_hidden)).to(device) # [num_layers(=1) * num_directions(=2), batch_size, n_hidden]
          cell_state = Variable(torch.zeros(1*2,  x.shape[1], n_hidden)).to(device) # [num_layers(=1) * num_directions(=2), batch_size, n_hidden]
          
          # final_hidden_state, final_cell_state : [num_layers(=1) * num_directions(=2), batch_size, n_hidden]
          #output, final_hidden_state = self.lstm(x)
          #output, (final_hidden_state, final_cell_state) = self.lstm(x)
          output, (final_hidden_state, final_cell_state) = self.lstm(x, (hidden_state, cell_state))
          #output = output.permute(1, 0, 2) # output : [batch_size, len_seq, n_hidden]
          #attn_output, attention = self.attention_net(output, final_hidden_state)
          #features = self.activation(self.encoder_fc(attn_output)) # model : [batch_size, num_classes]
          #x = self.emb(x)        
          #lstm_out, (h_n, c_n) = self.lstm(x)
          hidden_out = torch.cat((final_hidden_state[0,:,:],final_hidden_state[1,:,:]),1)
          #z = self.encoder_fc(hidden_out)
          return self.encoder_fc(hidden_out), self.emb

  from sklearn.metrics import accuracy_score
  n_hidden = 256
  n_emb = emb_dim
  total_epoch = 25
  model = Bi_LSTM_Attention().to(device)
  criterion = nn.CrossEntropyLoss()
  optimizer = optim.Adam(model.parameters(), lr=0.001)
  input_torch = torch.from_numpy(np.array(text_pad_encoded)).to(device)
  target_torch = torch.from_numpy(np.array(label_encoded)).view(-1).to(device)

  for epoch in range(total_epoch):  
      
      model.train()
        
      optimizer.zero_grad()
      
      outputs, lstm_emb = model(input_torch) 
      
      back_emb = {}
      
      for i, word in enumerate(word_list):
        back_emb[word] = lstm_emb(torch.LongTensor([i])).detach().numpy().flatten()
      _, _, total_acc, loss_1, loss_2 = word_analogy_evaluation(back_emb)
      avg_loss = 10.24*(total_acc+loss_1+loss_2)/3
      print("weight adjusted",avg_loss)
      loss = criterion(outputs, target_torch)-avg_loss
      loss.backward()
      optimizer.step()
        
      predicted = torch.argmax(outputs, -1)
      acc= accuracy_score(predicted.cpu().numpy(),target_torch.cpu().numpy())

      print('Epoch: %d, loss: %.5f, train_acc: %.2f' %(epoch + 1, loss.item(), acc))

  print('Finished Training')

  return back_emb

### update_embeddingx function
##### input: emb_table: an corpus_num*100 2-dim np array

#### version 0 of update_embedding function
##### loss function: nn.CE(predicted,expected)

In [ ]:
def update_embedding0(emb_table):
  # model for Text
  import torch
  import torch.nn as nn
  import torch.optim as optim
  from torch.autograd import Variable
  import torch.nn.functional as F

  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

  class Bi_LSTM_Attention(nn.Module):
      def __init__(self):
          super(Bi_LSTM_Attention, self).__init__()
          self.emb = nn.Embedding(emb_table.shape[0],emb_table.shape[1])
          self.emb.weight.data.copy_(torch.from_numpy(emb_table))
          self.emb.weight.requires_grad = True
          self.lstm = nn.LSTM(emb_table.shape[1], n_hidden, bidirectional=True)
          self.encoder_fc = nn.Linear(2*n_hidden, n_class)
          #self.activation = nn.ReLU()
      
      # https://colab.research.google.com/github/ngduyanhece/nlp-tutorial/blob/master/4-3.Bi-LSTM%28Attention%29/Bi_LSTM%28Attention%29_Torch.ipynb
      # output : [batch_size, n_step, n_hidden * num_directions(=2)], F matrix
      def attention_net(self, output, final_state):
          hidden = final_state.view(-1, n_hidden * 2, 1)   # hidden : [batch_size, n_hidden * num_directions(=2), 1(=n_layer)]
          attn_weights = torch.bmm(output, hidden).squeeze(2) # attn_weights : [batch_size, n_step]
          soft_attn_weights = F.softmax(attn_weights, 1).to(device)
          # [batch_size, n_hidden * num_directions(=2), n_step] * [batch_size, n_step, 1] = [batch_size, n_hidden * num_directions(=2), 1]
          context = torch.bmm(output.transpose(1, 2), soft_attn_weights.unsqueeze(2)).squeeze(2).to(device)
          #return context, soft_attn_weights # context : [batch_size, n_hidden * num_directions(=2)]
          return context, soft_attn_weights.data.numpy() # context : [batch_size, n_hidden * num_directions(=2)]

      def forward(self, X):
          x = self.emb(X) # input : [batch_size, len_seq, embedding_dim]
          # 32, 28, 100
          x = x.permute(1, 0, 2) # input : [len_seq, batch_size, embedding_dim]
          hidden_state = Variable(torch.zeros(1*2, x.shape[1], n_hidden)).to(device) # [num_layers(=1) * num_directions(=2), batch_size, n_hidden]
          cell_state = Variable(torch.zeros(1*2,  x.shape[1], n_hidden)).to(device) # [num_layers(=1) * num_directions(=2), batch_size, n_hidden]
          
          # final_hidden_state, final_cell_state : [num_layers(=1) * num_directions(=2), batch_size, n_hidden]
          #output, final_hidden_state = self.lstm(x)
          #output, (final_hidden_state, final_cell_state) = self.lstm(x)
          output, (final_hidden_state, final_cell_state) = self.lstm(x, (hidden_state, cell_state))
          #output = output.permute(1, 0, 2) # output : [batch_size, len_seq, n_hidden]
          #attn_output, attention = self.attention_net(output, final_hidden_state)
          #features = self.activation(self.encoder_fc(attn_output)) # model : [batch_size, num_classes]
          #x = self.emb(x)        
          #lstm_out, (h_n, c_n) = self.lstm(x)
          hidden_out = torch.cat((final_hidden_state[0,:,:],final_hidden_state[1,:,:]),1)
          #z = self.encoder_fc(hidden_out)
          return self.encoder_fc(hidden_out), self.emb

  from sklearn.metrics import accuracy_score
  n_hidden = 256
  n_emb = emb_dim
  total_epoch = 25
  model = Bi_LSTM_Attention().to(device)
  criterion = nn.CrossEntropyLoss()
  optimizer = optim.Adam(model.parameters(), lr=0.001)
  input_torch = torch.from_numpy(np.array(text_pad_encoded)).to(device)
  target_torch = torch.from_numpy(np.array(label_encoded)).view(-1).to(device)

  for epoch in range(total_epoch):  
      
      model.train()
        
      optimizer.zero_grad()
      
      outputs, lstm_emb = model(input_torch) 
      
      back_emb = {}
      
      for i, word in enumerate(word_list):
        back_emb[word] = lstm_emb(torch.LongTensor([i])).detach().numpy().flatten()
      _, _, total_acc, loss_1, loss_2 = word_analogy_evaluation(back_emb)
      avg_loss = 10.24*(total_acc+loss_1+loss_2)/3
      print("weight adjusted",avg_loss)
      loss = criterion(outputs, target_torch)
      loss.backward()
      optimizer.step()
        
      predicted = torch.argmax(outputs, -1)
      acc= accuracy_score(predicted.cpu().numpy(),target_torch.cpu().numpy())

      print('Epoch: %d, loss: %.5f, train_acc: %.2f' %(epoch + 1, loss.item(), acc))

  print('Finished Training')

  return back_emb

#### version 1 of update_embedding function
##### loss function: nn.CE(predicted,expected)+Spearmans loss

In [ ]:
def update_embedding1(emb_table):
  # model for Text
  import torch
  import torch.nn as nn
  import torch.optim as optim
  from torch.autograd import Variable
  import torch.nn.functional as F

  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

  class Bi_LSTM_Attention(nn.Module):
      def __init__(self):
          super(Bi_LSTM_Attention, self).__init__()
          self.emb = nn.Embedding(emb_table.shape[0],emb_table.shape[1])
          self.emb.weight.data.copy_(torch.from_numpy(emb_table))
          self.emb.weight.requires_grad = True
          self.lstm = nn.LSTM(emb_table.shape[1], n_hidden, bidirectional=True)
          self.encoder_fc = nn.Linear(2*n_hidden, n_class)
          #self.activation = nn.ReLU()
      
      # https://colab.research.google.com/github/ngduyanhece/nlp-tutorial/blob/master/4-3.Bi-LSTM%28Attention%29/Bi_LSTM%28Attention%29_Torch.ipynb
      # output : [batch_size, n_step, n_hidden * num_directions(=2)], F matrix
      def attention_net(self, output, final_state):
          hidden = final_state.view(-1, n_hidden * 2, 1)   # hidden : [batch_size, n_hidden * num_directions(=2), 1(=n_layer)]
          attn_weights = torch.bmm(output, hidden).squeeze(2) # attn_weights : [batch_size, n_step]
          soft_attn_weights = F.softmax(attn_weights, 1).to(device)
          # [batch_size, n_hidden * num_directions(=2), n_step] * [batch_size, n_step, 1] = [batch_size, n_hidden * num_directions(=2), 1]
          context = torch.bmm(output.transpose(1, 2), soft_attn_weights.unsqueeze(2)).squeeze(2).to(device)
          #return context, soft_attn_weights # context : [batch_size, n_hidden * num_directions(=2)]
          return context, soft_attn_weights.data.numpy() # context : [batch_size, n_hidden * num_directions(=2)]

      def forward(self, X):
          x = self.emb(X) # input : [batch_size, len_seq, embedding_dim]
          # 32, 28, 100
          x = x.permute(1, 0, 2) # input : [len_seq, batch_size, embedding_dim]
          hidden_state = Variable(torch.zeros(1*2, x.shape[1], n_hidden)).to(device) # [num_layers(=1) * num_directions(=2), batch_size, n_hidden]
          cell_state = Variable(torch.zeros(1*2,  x.shape[1], n_hidden)).to(device) # [num_layers(=1) * num_directions(=2), batch_size, n_hidden]
          
          # final_hidden_state, final_cell_state : [num_layers(=1) * num_directions(=2), batch_size, n_hidden]
          #output, final_hidden_state = self.lstm(x)
          #output, (final_hidden_state, final_cell_state) = self.lstm(x)
          output, (final_hidden_state, final_cell_state) = self.lstm(x, (hidden_state, cell_state))
          #output = output.permute(1, 0, 2) # output : [batch_size, len_seq, n_hidden]
          #attn_output, attention = self.attention_net(output, final_hidden_state)
          #features = self.activation(self.encoder_fc(attn_output)) # model : [batch_size, num_classes]
          #x = self.emb(x)        
          #lstm_out, (h_n, c_n) = self.lstm(x)
          hidden_out = torch.cat((final_hidden_state[0,:,:],final_hidden_state[1,:,:]),1)
          #z = self.encoder_fc(hidden_out)
          return self.encoder_fc(hidden_out), self.emb

  from sklearn.metrics import accuracy_score
  n_hidden = 256
  n_emb = emb_dim
  total_epoch = 25
  model = Bi_LSTM_Attention().to(device)
  criterion = nn.CrossEntropyLoss()
  optimizer = optim.Adam(model.parameters(), lr=0.001)
  input_torch = torch.from_numpy(np.array(text_pad_encoded)).to(device)
  target_torch = torch.from_numpy(np.array(label_encoded)).view(-1).to(device)

  for epoch in range(total_epoch):  
      
      model.train()
        
      optimizer.zero_grad()
      
      outputs, lstm_emb = model(input_torch) 
      
      back_emb = {}
      
      for i, word in enumerate(word_list):
        back_emb[word] = lstm_emb(torch.LongTensor([i])).detach().numpy().flatten()
      _, _, total_acc, loss_1, loss_2 = word_analogy_evaluation(back_emb)
      sim_loss = 33*sim_loss_cal2(word_sim_dir,back_emb)
      print("sparsemans loss",sim_loss)
      loss = criterion(outputs, target_torch)+sim_loss
      loss.backward()
      optimizer.step()
        
      predicted = torch.argmax(outputs, -1)
      acc= accuracy_score(predicted.cpu().numpy(),target_torch.cpu().numpy())

      print('Epoch: %d, loss: %.5f, train_acc: %.2f' %(epoch + 1, loss.item(), acc))

  print('Finished Training')

  return back_emb

#### version 2 of update_embedding function
##### loss function: nn.CE(predicted,expected) + analogy_loss (avg of 3)

In [ ]:
def update_embedding2(emb_table):
  # model for Text
  import torch
  import torch.nn as nn
  import torch.optim as optim
  from torch.autograd import Variable
  import torch.nn.functional as F

  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

  class Bi_LSTM_Attention(nn.Module):
      def __init__(self):
          super(Bi_LSTM_Attention, self).__init__()
          self.emb = nn.Embedding(emb_table.shape[0],emb_table.shape[1])
          self.emb.weight.data.copy_(torch.from_numpy(emb_table))
          self.emb.weight.requires_grad = True
          self.lstm = nn.LSTM(emb_table.shape[1], n_hidden, bidirectional=True)
          self.encoder_fc = nn.Linear(2*n_hidden, n_class)
          #self.activation = nn.ReLU()
      
      # https://colab.research.google.com/github/ngduyanhece/nlp-tutorial/blob/master/4-3.Bi-LSTM%28Attention%29/Bi_LSTM%28Attention%29_Torch.ipynb
      # output : [batch_size, n_step, n_hidden * num_directions(=2)], F matrix
      def attention_net(self, output, final_state):
          hidden = final_state.view(-1, n_hidden * 2, 1)   # hidden : [batch_size, n_hidden * num_directions(=2), 1(=n_layer)]
          attn_weights = torch.bmm(output, hidden).squeeze(2) # attn_weights : [batch_size, n_step]
          soft_attn_weights = F.softmax(attn_weights, 1).to(device)
          # [batch_size, n_hidden * num_directions(=2), n_step] * [batch_size, n_step, 1] = [batch_size, n_hidden * num_directions(=2), 1]
          context = torch.bmm(output.transpose(1, 2), soft_attn_weights.unsqueeze(2)).squeeze(2).to(device)
          #return context, soft_attn_weights # context : [batch_size, n_hidden * num_directions(=2)]
          return context, soft_attn_weights.data.numpy() # context : [batch_size, n_hidden * num_directions(=2)]

      def forward(self, X):
          x = self.emb(X) # input : [batch_size, len_seq, embedding_dim]
          # 32, 28, 100
          x = x.permute(1, 0, 2) # input : [len_seq, batch_size, embedding_dim]
          hidden_state = Variable(torch.zeros(1*2, x.shape[1], n_hidden)).to(device) # [num_layers(=1) * num_directions(=2), batch_size, n_hidden]
          cell_state = Variable(torch.zeros(1*2,  x.shape[1], n_hidden)).to(device) # [num_layers(=1) * num_directions(=2), batch_size, n_hidden]
          
          # final_hidden_state, final_cell_state : [num_layers(=1) * num_directions(=2), batch_size, n_hidden]
          #output, final_hidden_state = self.lstm(x)
          #output, (final_hidden_state, final_cell_state) = self.lstm(x)
          output, (final_hidden_state, final_cell_state) = self.lstm(x, (hidden_state, cell_state))
          #output = output.permute(1, 0, 2) # output : [batch_size, len_seq, n_hidden]
          #attn_output, attention = self.attention_net(output, final_hidden_state)
          #features = self.activation(self.encoder_fc(attn_output)) # model : [batch_size, num_classes]
          #x = self.emb(x)        
          #lstm_out, (h_n, c_n) = self.lstm(x)
          hidden_out = torch.cat((final_hidden_state[0,:,:],final_hidden_state[1,:,:]),1)
          #z = self.encoder_fc(hidden_out)
          return self.encoder_fc(hidden_out), self.emb

  from sklearn.metrics import accuracy_score
  n_hidden = 256
  n_emb = emb_dim
  total_epoch = 25
  model = Bi_LSTM_Attention().to(device)
  criterion = nn.CrossEntropyLoss()
  optimizer = optim.Adam(model.parameters(), lr=0.001)
  input_torch = torch.from_numpy(np.array(text_pad_encoded)).to(device)
  target_torch = torch.from_numpy(np.array(label_encoded)).view(-1).to(device)

  for epoch in range(total_epoch):  
      
      model.train()
        
      optimizer.zero_grad()
      
      outputs, lstm_emb = model(input_torch) 
      
      back_emb = {}
      
      for i, word in enumerate(word_list):
        back_emb[word] = lstm_emb(torch.LongTensor([i])).detach().numpy().flatten()
      _, _, total_acc, loss_1, loss_2 = word_analogy_evaluation(back_emb)
      avg_loss = 0.1024*(total_acc+loss_1+loss_2)/3
      print("weight adjusted",avg_loss)
      loss = criterion(outputs, target_torch)-avg_loss
      loss.backward()
      optimizer.step()
        
      predicted = torch.argmax(outputs, -1)
      acc= accuracy_score(predicted.cpu().numpy(),target_torch.cpu().numpy())

      print('Epoch: %d, loss: %.5f, train_acc: %.2f' %(epoch + 1, loss.item(), acc))

  print('Finished Training')

  return back_emb

#### version 3 of update_embedding function
##### loss function: nn.CE(predicted,expected) + sparsemans loss + analogy_loss (avg of 3)

In [ ]:
def update_embedding3(emb_table):
  # model for Text
  import torch
  import torch.nn as nn
  import torch.optim as optim
  from torch.autograd import Variable
  import torch.nn.functional as F

  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

  class Bi_LSTM_Attention(nn.Module):
      def __init__(self):
          super(Bi_LSTM_Attention, self).__init__()
          self.emb = nn.Embedding(emb_table.shape[0],emb_table.shape[1])
          self.emb.weight.data.copy_(torch.from_numpy(emb_table))
          self.emb.weight.requires_grad = True
          self.lstm = nn.LSTM(emb_table.shape[1], n_hidden, bidirectional=True)
          self.encoder_fc = nn.Linear(2*n_hidden, n_class)
          #self.activation = nn.ReLU()
      
      # https://colab.research.google.com/github/ngduyanhece/nlp-tutorial/blob/master/4-3.Bi-LSTM%28Attention%29/Bi_LSTM%28Attention%29_Torch.ipynb
      # output : [batch_size, n_step, n_hidden * num_directions(=2)], F matrix
      def attention_net(self, output, final_state):
          hidden = final_state.view(-1, n_hidden * 2, 1)   # hidden : [batch_size, n_hidden * num_directions(=2), 1(=n_layer)]
          attn_weights = torch.bmm(output, hidden).squeeze(2) # attn_weights : [batch_size, n_step]
          soft_attn_weights = F.softmax(attn_weights, 1).to(device)
          # [batch_size, n_hidden * num_directions(=2), n_step] * [batch_size, n_step, 1] = [batch_size, n_hidden * num_directions(=2), 1]
          context = torch.bmm(output.transpose(1, 2), soft_attn_weights.unsqueeze(2)).squeeze(2).to(device)
          #return context, soft_attn_weights # context : [batch_size, n_hidden * num_directions(=2)]
          return context, soft_attn_weights.data.numpy() # context : [batch_size, n_hidden * num_directions(=2)]

      def forward(self, X):
          x = self.emb(X) # input : [batch_size, len_seq, embedding_dim]
          # 32, 28, 100
          x = x.permute(1, 0, 2) # input : [len_seq, batch_size, embedding_dim]
          hidden_state = Variable(torch.zeros(1*2, x.shape[1], n_hidden)).to(device) # [num_layers(=1) * num_directions(=2), batch_size, n_hidden]
          cell_state = Variable(torch.zeros(1*2,  x.shape[1], n_hidden)).to(device) # [num_layers(=1) * num_directions(=2), batch_size, n_hidden]
          
          # final_hidden_state, final_cell_state : [num_layers(=1) * num_directions(=2), batch_size, n_hidden]
          #output, final_hidden_state = self.lstm(x)
          #output, (final_hidden_state, final_cell_state) = self.lstm(x)
          output, (final_hidden_state, final_cell_state) = self.lstm(x, (hidden_state, cell_state))
          #output = output.permute(1, 0, 2) # output : [batch_size, len_seq, n_hidden]
          #attn_output, attention = self.attention_net(output, final_hidden_state)
          #features = self.activation(self.encoder_fc(attn_output)) # model : [batch_size, num_classes]
          #x = self.emb(x)        
          #lstm_out, (h_n, c_n) = self.lstm(x)
          hidden_out = torch.cat((final_hidden_state[0,:,:],final_hidden_state[1,:,:]),1)
          #z = self.encoder_fc(hidden_out)
          return self.encoder_fc(hidden_out), self.emb

  from sklearn.metrics import accuracy_score
  n_hidden = 256
  n_emb = emb_dim
  total_epoch = 25
  model = Bi_LSTM_Attention().to(device)
  criterion = nn.CrossEntropyLoss()
  optimizer = optim.Adam(model.parameters(), lr=0.001)
  input_torch = torch.from_numpy(np.array(text_pad_encoded)).to(device)
  target_torch = torch.from_numpy(np.array(label_encoded)).view(-1).to(device)

  for epoch in range(total_epoch):  
      
      model.train()
        
      optimizer.zero_grad()
      
      outputs, lstm_emb = model(input_torch) 
      
      back_emb = {}
      
      for i, word in enumerate(word_list):
        back_emb[word] = lstm_emb(torch.LongTensor([i])).detach().numpy().flatten()
      _, _, total_acc, loss_1, loss_2 = word_analogy_evaluation(back_emb)
      sim_loss = 3.3*sim_loss_cal2(word_sim_dir,back_emb)
      print("sparsemans loss",sim_loss)
      avg_loss = 10.24*(total_acc+loss_1+loss_2)/3
      print("word analogy weight adjusted",avg_loss)
      loss = criterion(outputs, target_torch)-avg_loss+sim_loss
      loss.backward()
      optimizer.step()
        
      predicted = torch.argmax(outputs, -1)
      acc= accuracy_score(predicted.cpu().numpy(),target_torch.cpu().numpy())

      print('Epoch: %d, loss: %.5f, train_acc: %.2f' %(epoch + 1, loss.item(), acc))

  print('Finished Training')

  return back_emb

#### Version 4 of update embedding function
Attention

In [ ]:
def update_embedding(emb_table):
  # model for Text
  import torch
  import torch.nn as nn
  import torch.optim as optim
  from torch.autograd import Variable
  import torch.nn.functional as F

  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

  class Bi_LSTM_Attention(nn.Module):
      def __init__(self):
          super(Bi_LSTM_Attention, self).__init__()
          self.emb = nn.Embedding(emb_table.shape[0],emb_table.shape[1])
          self.emb.weight.data.copy_(torch.from_numpy(emb_table))
          self.emb.weight.requires_grad = True
          self.lstm = nn.LSTM(emb_table.shape[1], n_hidden, bidirectional=True)
          self.encoder_fc = nn.Linear(2*n_hidden, n_class)
          #self.activation = nn.ReLU()
      
      # https://colab.research.google.com/github/ngduyanhece/nlp-tutorial/blob/master/4-3.Bi-LSTM%28Attention%29/Bi_LSTM%28Attention%29_Torch.ipynb
      # output : [batch_size, n_step, n_hidden * num_directions(=2)], F matrix
      def attention_net(self, output, final_state):
          hidden = final_state.view(-1, n_hidden * 2, 1)   # hidden : [batch_size, n_hidden * num_directions(=2), 1(=n_layer)]
          attn_weights = torch.bmm(output, hidden).squeeze(2) # attn_weights : [batch_size, n_step]
          soft_attn_weights = F.softmax(attn_weights, 1).to(device)
          # [batch_size, n_hidden * num_directions(=2), n_step] * [batch_size, n_step, 1] = [batch_size, n_hidden * num_directions(=2), 1]
          context = torch.bmm(output.transpose(1, 2), soft_attn_weights.unsqueeze(2)).squeeze(2).to(device)
          #return context, soft_attn_weights # context : [batch_size, n_hidden * num_directions(=2)]
          return context, soft_attn_weights.data.numpy() # context : [batch_size, n_hidden * num_directions(=2)]

      def forward(self, X):
          x = self.emb(X) # input : [batch_size, len_seq, embedding_dim]
          # 32, 28, 100
          x = x.permute(1, 0, 2) # input : [len_seq, batch_size, embedding_dim]
          hidden_state = Variable(torch.zeros(1*2, x.shape[1], n_hidden)).to(device) # [num_layers(=1) * num_directions(=2), batch_size, n_hidden]
          cell_state = Variable(torch.zeros(1*2,  x.shape[1], n_hidden)).to(device) # [num_layers(=1) * num_directions(=2), batch_size, n_hidden]
          
          # final_hidden_state, final_cell_state : [num_layers(=1) * num_directions(=2), batch_size, n_hidden]
          #output, final_hidden_state = self.lstm(x)
          #output, (final_hidden_state, final_cell_state) = self.lstm(x)
          output, (final_hidden_state, final_cell_state) = self.lstm(x, (hidden_state, cell_state))
          output = output.permute(1, 0, 2) # output : [batch_size, len_seq, n_hidden]
          attn_output, attention = self.attention_net(output, final_hidden_state)
          features = self.activation(self.encoder_fc(attn_output)) # model : [batch_size, num_classes]
          #x = self.emb(x)        
          #lstm_out, (h_n, c_n) = self.lstm(x)
          hidden_out = torch.cat((final_hidden_state[0,:,:],final_hidden_state[1,:,:]),1)
          #z = self.encoder_fc(hidden_out)
          return self.encoder_fc(hidden_out), self.emb

          # final_hidden_state, final_cell_state : [num_layers(=1) * num_directions(=2), batch_size, n_hidden]
          output, (final_hidden_state, final_cell_state) = self.lstm(input, (hidden_state, cell_state))
          output = output.permute(1, 0, 2) # output : [batch_size, len_seq, n_hidden]
          attn_output, attention = self.attention_net(output, final_hidden_state)
          return self.out(attn_output), attention # model : [batch_size, num_classes], attention : [batch_size, n_step]

  from sklearn.metrics import accuracy_score
  n_hidden = 256
  n_emb = emb_dim
  total_epoch = 25
  model = Bi_LSTM_Attention().to(device)
  criterion = nn.CrossEntropyLoss()
  optimizer = optim.Adam(model.parameters(), lr=0.001)
  input_torch = torch.from_numpy(np.array(text_pad_encoded)).to(device)
  target_torch = torch.from_numpy(np.array(label_encoded)).view(-1).to(device)

  for epoch in range(total_epoch):  
      
      model.train()
        
      optimizer.zero_grad()
      
      outputs, lstm_emb = model(input_torch) 
      
      back_emb = {}
      
      for i, word in enumerate(word_list):
        back_emb[word] = lstm_emb(torch.LongTensor([i])).detach().numpy().flatten()
      _, _, total_acc, loss_1, loss_2 = word_analogy_evaluation(back_emb)
      avg_loss = 10.24*(total_acc+loss_1+loss_2)/3
      print("weight adjusted",avg_loss)
      loss = criterion(outputs, target_torch)-avg_loss
      loss.backward()
      optimizer.step()
        
      predicted = torch.argmax(outputs, -1)
      acc= accuracy_score(predicted.cpu().numpy(),target_torch.cpu().numpy())

      print('Epoch: %d, loss: %.5f, train_acc: %.2f' %(epoch + 1, loss.item(), acc))

  print('Finished Training')

  return back_emb

### Classificatin Fuction
##### 1. Tune the word embedding on the train set
##### 2. Do 2-class classification on the test set

### For classification methods, we design two types of NN:
##### 1. BiLSTM

##### 1. BiLSTM for classification

In [ ]:
def classification(emb_table):
  # model for Text
  import torch
  import torch.nn as nn
  import torch.optim as optim
  from torch.autograd import Variable
  import torch.nn.functional as F

  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

  class Bi_LSTM_Attention(nn.Module):
      def __init__(self):
          super(Bi_LSTM_Attention, self).__init__()
          self.emb = nn.Embedding(emb_table.shape[0],emb_table.shape[1])
          self.emb.weight.data.copy_(torch.from_numpy(emb_table))
          self.emb.weight.requires_grad = False
          self.lstm = nn.LSTM(emb_table.shape[1], n_hidden, bidirectional=True)
          self.encoder_fc = nn.Linear(2*n_hidden, n_class)
          #self.activation = nn.ReLU()
      
      # https://colab.research.google.com/github/ngduyanhece/nlp-tutorial/blob/master/4-3.Bi-LSTM%28Attention%29/Bi_LSTM%28Attention%29_Torch.ipynb
      # output : [batch_size, n_step, n_hidden * num_directions(=2)], F matrix
      def attention_net(self, output, final_state):
          hidden = final_state.view(-1, n_hidden * 2, 1)   # hidden : [batch_size, n_hidden * num_directions(=2), 1(=n_layer)]
          attn_weights = torch.bmm(output, hidden).squeeze(2) # attn_weights : [batch_size, n_step]
          soft_attn_weights = F.softmax(attn_weights, 1).to(device)
          # [batch_size, n_hidden * num_directions(=2), n_step] * [batch_size, n_step, 1] = [batch_size, n_hidden * num_directions(=2), 1]
          context = torch.bmm(output.transpose(1, 2), soft_attn_weights.unsqueeze(2)).squeeze(2).to(device)
          #return context, soft_attn_weights # context : [batch_size, n_hidden * num_directions(=2)]
          return context, soft_attn_weights.data.numpy() # context : [batch_size, n_hidden * num_directions(=2)]

      def forward(self, X):
          x = self.emb(X) # input : [batch_size, len_seq, embedding_dim]
          # 32, 28, 100
          x = x.permute(1, 0, 2) # input : [len_seq, batch_size, embedding_dim]
          hidden_state = Variable(torch.zeros(1*2, x.shape[1], n_hidden)).to(device) # [num_layers(=1) * num_directions(=2), batch_size, n_hidden]
          cell_state = Variable(torch.zeros(1*2,  x.shape[1], n_hidden)).to(device) # [num_layers(=1) * num_directions(=2), batch_size, n_hidden]
          
          # final_hidden_state, final_cell_state : [num_layers(=1) * num_directions(=2), batch_size, n_hidden]
          #output, final_hidden_state = self.lstm(x)
          #output, (final_hidden_state, final_cell_state) = self.lstm(x)
          output, (final_hidden_state, final_cell_state) = self.lstm(x, (hidden_state, cell_state))
          #output = output.permute(1, 0, 2) # output : [batch_size, len_seq, n_hidden]
          #attn_output, attention = self.attention_net(output, final_hidden_state)
          #features = self.activation(self.encoder_fc(attn_output)) # model : [batch_size, num_classes]
          #x = self.emb(x)        sf
          #lstm_out, (h_n, c_n) = self.lstm(x)
          hidden_out = torch.cat((final_hidden_state[0,:,:],final_hidden_state[1,:,:]),1)
          #z = self.encoder_fc(hidden_out)
          return self.encoder_fc(hidden_out)

  from sklearn.metrics import accuracy_score
  n_hidden = 256
  n_emb = emb_dim
  total_epoch = 25
  model = Bi_LSTM_Attention().to(device)
  criterion = nn.CrossEntropyLoss()
  optimizer = optim.Adam(model.parameters(), lr=0.005)
  input_torch = torch.from_numpy(np.array(train_pad_encoded)).to(device)
  target_torch = torch.from_numpy(np.array(label_train_encoded)).view(-1).to(device)

  for epoch in range(total_epoch):  
      
      model.train()
        
      optimizer.zero_grad()
      
      outputs = model(input_torch) 
      loss = criterion(outputs, target_torch) 
      loss.backward()
      optimizer.step()
        
      predicted = torch.argmax(outputs, -1)
      acc= accuracy_score(predicted.cpu().numpy(),target_torch.cpu().numpy())

      print('Epoch: %d, loss: %.5f, train_acc: %.2f' %(epoch + 1, loss.item(), acc))

  print('Finished Training')

  model.eval()

  input_torch = torch.from_numpy(np.array(test_pad_encoded)).to(device)

  outputs = model(input_torch) 
  predicted = torch.argmax(outputs, -1)

  from sklearn.metrics import classification_report
  print(classification_report(label_test_encoded,predicted.cpu().numpy()))

In [ ]:
def classification_lr(emb_table,learning_rate):
  # model for Text
  import torch
  import torch.nn as nn
  import torch.optim as optim
  from torch.autograd import Variable
  import torch.nn.functional as F

  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

  class Bi_LSTM_Attention(nn.Module):
      def __init__(self):
          super(Bi_LSTM_Attention, self).__init__()
          self.emb = nn.Embedding(emb_table.shape[0],emb_table.shape[1])
          self.emb.weight.data.copy_(torch.from_numpy(emb_table))
          self.emb.weight.requires_grad = False
          self.lstm = nn.LSTM(emb_table.shape[1], n_hidden, bidirectional=True)
          self.encoder_fc = nn.Linear(2*n_hidden, n_class)
          #self.activation = nn.ReLU()
      
      # https://colab.research.google.com/github/ngduyanhece/nlp-tutorial/blob/master/4-3.Bi-LSTM%28Attention%29/Bi_LSTM%28Attention%29_Torch.ipynb
      # output : [batch_size, n_step, n_hidden * num_directions(=2)], F matrix
      def attention_net(self, output, final_state):
          hidden = final_state.view(-1, n_hidden * 2, 1)   # hidden : [batch_size, n_hidden * num_directions(=2), 1(=n_layer)]
          attn_weights = torch.bmm(output, hidden).squeeze(2) # attn_weights : [batch_size, n_step]
          soft_attn_weights = F.softmax(attn_weights, 1).to(device)
          # [batch_size, n_hidden * num_directions(=2), n_step] * [batch_size, n_step, 1] = [batch_size, n_hidden * num_directions(=2), 1]
          context = torch.bmm(output.transpose(1, 2), soft_attn_weights.unsqueeze(2)).squeeze(2).to(device)
          #return context, soft_attn_weights # context : [batch_size, n_hidden * num_directions(=2)]
          return context, soft_attn_weights.data.numpy() # context : [batch_size, n_hidden * num_directions(=2)]

      def forward(self, X):
          x = self.emb(X) # input : [batch_size, len_seq, embedding_dim]
          # 32, 28, 100
          x = x.permute(1, 0, 2) # input : [len_seq, batch_size, embedding_dim]
          hidden_state = Variable(torch.zeros(1*2, x.shape[1], n_hidden)).to(device) # [num_layers(=1) * num_directions(=2), batch_size, n_hidden]
          cell_state = Variable(torch.zeros(1*2,  x.shape[1], n_hidden)).to(device) # [num_layers(=1) * num_directions(=2), batch_size, n_hidden]
          
          # final_hidden_state, final_cell_state : [num_layers(=1) * num_directions(=2), batch_size, n_hidden]
          #output, final_hidden_state = self.lstm(x)
          #output, (final_hidden_state, final_cell_state) = self.lstm(x)
          output, (final_hidden_state, final_cell_state) = self.lstm(x, (hidden_state, cell_state))
          #output = output.permute(1, 0, 2) # output : [batch_size, len_seq, n_hidden]
          #attn_output, attention = self.attention_net(output, final_hidden_state)
          #features = self.activation(self.encoder_fc(attn_output)) # model : [batch_size, num_classes]
          #x = self.emb(x)        
          #lstm_out, (h_n, c_n) = self.lstm(x)
          hidden_out = torch.cat((final_hidden_state[0,:,:],final_hidden_state[1,:,:]),1)
          #z = self.encoder_fc(hidden_out)
          return self.encoder_fc(hidden_out)

  from sklearn.metrics import accuracy_score
  n_hidden = 32#256
  n_emb = emb_dim
  total_epoch = 25
  model = Bi_LSTM_Attention().to(device)
  criterion = nn.CrossEntropyLoss()
  optimizer = optim.Adam(model.parameters(), lr=learning_rate)
  input_torch = torch.from_numpy(np.array(train_pad_encoded)).to(device)
  target_torch = torch.from_numpy(np.array(label_train_encoded)).view(-1).to(device)

  for epoch in range(total_epoch):  
      
      model.train()
        
      optimizer.zero_grad()
      
      outputs = model(input_torch) 
      loss = criterion(outputs, target_torch) 
      loss.backward()
      optimizer.step()
        
      predicted = torch.argmax(outputs, -1)
      acc= accuracy_score(predicted.cpu().numpy(),target_torch.cpu().numpy())

      print('Epoch: %d, loss: %.5f, train_acc: %.2f' %(epoch + 1, loss.item(), acc))

  print('Finished Training')

  model.eval()

  input_torch = torch.from_numpy(np.array(test_pad_encoded)).to(device)

  outputs = model(input_torch) 
  predicted = torch.argmax(outputs, -1)

  from sklearn.metrics import classification_report
  print(classification_report(label_test_encoded,predicted.cpu().numpy()))

Load the Dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
data = pd.read_csv("/content/drive/MyDrive/IMDB Dataset.csv")
data = data.sample(1000,random_state=24)

print("Size of the dataset: {0}".format(len(data)))

Size of the dataset: 1000


In [ ]:
text = data["review"].tolist()
label = data["sentiment"].tolist()

print(text[1:10])
print(label[1:10])

["This film did entertain me with lots of laughs at the actors who kept the film moving along in all types of crazy directions. If you like suggestive language and sexy looking gals they were all in the picture and gals and guys all looking burned out before they even graduate from high school. There is one scene where the teenagers drive their car into a very fake deer and then proceed to throw it out into a lake or ocean, which is repeated over and over again. There is no horror to this film except the word Horrible for the entire picture and Arnold who plays a plastic cop is really one sick character. Please don't waste your time viewing this film.", 'The animation in this re-imagining of Peter & the Wolf is excellent, but at 29 minutes, the film is sleep inducing. They should have called it "Peter & the Snails", because everything moves at a snail\'s pace. I couldn\'t even watch the film in one sitting - I had to watch it 15 minutes at a time, and it was pure torture.<br /><br />Sa

In [ ]:
import re

def remove_clean(x):
    x = re.sub(r'<br /><br />','',x)
    x = x.lower()
    x = re.sub(r'[^\w\s]','',x)
    return x

text_clean = [remove_clean(s) for s in text]

print(text_clean[1:10])

['this film did entertain me with lots of laughs at the actors who kept the film moving along in all types of crazy directions if you like suggestive language and sexy looking gals they were all in the picture and gals and guys all looking burned out before they even graduate from high school there is one scene where the teenagers drive their car into a very fake deer and then proceed to throw it out into a lake or ocean which is repeated over and over again there is no horror to this film except the word horrible for the entire picture and arnold who plays a plastic cop is really one sick character please dont waste your time viewing this film', 'the animation in this reimagining of peter  the wolf is excellent but at 29 minutes the film is sleep inducing they should have called it peter  the snails because everything moves at a snails pace i couldnt even watch the film in one sitting  i had to watch it 15 minutes at a time and it was pure torturesave yourself 30 minutes  do not watch

In [ ]:
import nltk
nltk.download('punkt')

from nltk.tokenize import word_tokenize

text_clean = [word_tokenize(sentence) for sentence in text_clean]

print(text_clean[1:10])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


[['this', 'film', 'did', 'entertain', 'me', 'with', 'lots', 'of', 'laughs', 'at', 'the', 'actors', 'who', 'kept', 'the', 'film', 'moving', 'along', 'in', 'all', 'types', 'of', 'crazy', 'directions', 'if', 'you', 'like', 'suggestive', 'language', 'and', 'sexy', 'looking', 'gals', 'they', 'were', 'all', 'in', 'the', 'picture', 'and', 'gals', 'and', 'guys', 'all', 'looking', 'burned', 'out', 'before', 'they', 'even', 'graduate', 'from', 'high', 'school', 'there', 'is', 'one', 'scene', 'where', 'the', 'teenagers', 'drive', 'their', 'car', 'into', 'a', 'very', 'fake', 'deer', 'and', 'then', 'proceed', 'to', 'throw', 'it', 'out', 'into', 'a', 'lake', 'or', 'ocean', 'which', 'is', 'repeated', 'over', 'and', 'over', 'again', 'there', 'is', 'no', 'horror', 'to', 'this', 'film', 'except', 'the', 'word', 'horrible', 'for', 'the', 'entire', 'picture', 'and', 'arnold', 'who', 'plays', 'a', 'plastic', 'cop', 'is', 'really', 'one', 'sick', 'character', 'please', 'dont', 'waste', 'your', 'time', 'view

In [ ]:
nltk.download('stopwords')
from nltk.corpus import stopwords as sw
stop_words = sw.words('english')

text_cleaned =[]
for tokens in text_clean:
    filtered_sentence = [w for w in tokens if not w in stop_words]
    text_cleaned.append(filtered_sentence)

print(text_cleaned[1:10])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


[['film', 'entertain', 'lots', 'laughs', 'actors', 'kept', 'film', 'moving', 'along', 'types', 'crazy', 'directions', 'like', 'suggestive', 'language', 'sexy', 'looking', 'gals', 'picture', 'gals', 'guys', 'looking', 'burned', 'even', 'graduate', 'high', 'school', 'one', 'scene', 'teenagers', 'drive', 'car', 'fake', 'deer', 'proceed', 'throw', 'lake', 'ocean', 'repeated', 'horror', 'film', 'except', 'word', 'horrible', 'entire', 'picture', 'arnold', 'plays', 'plastic', 'cop', 'really', 'one', 'sick', 'character', 'please', 'dont', 'waste', 'time', 'viewing', 'film'], ['animation', 'reimagining', 'peter', 'wolf', 'excellent', '29', 'minutes', 'film', 'sleep', 'inducing', 'called', 'peter', 'snails', 'everything', 'moves', 'snails', 'pace', 'couldnt', 'even', 'watch', 'film', 'one', 'sitting', 'watch', '15', 'minutes', 'time', 'pure', 'torturesave', '30', 'minutes', 'watch', 'film', 'thank', 'mei', 'guess', 'oscar', 'nominating', 'committee', 'watched', 'first', 'minutes', 'nominees', 'u

In [ ]:
import nltk
nltk.download('wordnet')

nltk.download('omw-1.4')

from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

text_final = []
for tokens in text_cleaned:
    lemma_sentence = [lemmatizer.lemmatize(w) for w in tokens]
    text_final.append(lemma_sentence)

print(text_final[1:10])

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Unzipping corpora/omw-1.4.zip.


[['film', 'entertain', 'lot', 'laugh', 'actor', 'kept', 'film', 'moving', 'along', 'type', 'crazy', 'direction', 'like', 'suggestive', 'language', 'sexy', 'looking', 'gal', 'picture', 'gal', 'guy', 'looking', 'burned', 'even', 'graduate', 'high', 'school', 'one', 'scene', 'teenager', 'drive', 'car', 'fake', 'deer', 'proceed', 'throw', 'lake', 'ocean', 'repeated', 'horror', 'film', 'except', 'word', 'horrible', 'entire', 'picture', 'arnold', 'play', 'plastic', 'cop', 'really', 'one', 'sick', 'character', 'please', 'dont', 'waste', 'time', 'viewing', 'film'], ['animation', 'reimagining', 'peter', 'wolf', 'excellent', '29', 'minute', 'film', 'sleep', 'inducing', 'called', 'peter', 'snail', 'everything', 'move', 'snail', 'pace', 'couldnt', 'even', 'watch', 'film', 'one', 'sitting', 'watch', '15', 'minute', 'time', 'pure', 'torturesave', '30', 'minute', 'watch', 'film', 'thank', 'mei', 'guess', 'oscar', 'nominating', 'committee', 'watched', 'first', 'minute', 'nominee', 'unfortunately', 'vo

In [ ]:
import numpy as np

word_set = set() 
for sent in text_final:
  for word in sent:
    word_set.add(word)

word_set.add('[PAD]')
word_set.add('[UNKNOWN]')

word_list = list(word_set) 

word_list.sort()

word_index = {}
ind = 0
for word in word_list:
  word_index[word] = ind
  ind += 1

emb_dim = 100

### Text Encode

In [ ]:
def encode_and_add_padding(sentences, seq_length, word_index):
  sent_encoded = []
  for sent in sentences:
    temp_encoded = [word_index[word] if word in word_index else word_index['[UNKNOWN]'] for word in sent]
    if len(temp_encoded) < seq_length:
      temp_encoded += [word_index['[PAD]']] * (seq_length - len(temp_encoded))
    if len(temp_encoded) > seq_length:
      temp_encoded = temp_encoded[:seq_length]
    sent_encoded.append(temp_encoded)
  return sent_encoded

seq_length = 100
text_pad_encoded = encode_and_add_padding(text_final, seq_length, word_index)

from sklearn.preprocessing import LabelEncoder
import numpy as np

unique_labels = np.unique(label)

lEnc = LabelEncoder()
label_encoded = lEnc.fit_transform(label)

n_class = len(unique_labels)

In [ ]:
from sklearn.model_selection import train_test_split
train_pad_encoded,test_pad_encoded,label_train_encoded,label_test_encoded = train_test_split(text_pad_encoded,label_encoded,test_size=0.25,random_state=42)

### CBOW

In [ ]:
word2vec_cbow_emb = original_embedding(wv_cbow_model.wv)

In [ ]:
original_word2vec_cbow_acc = classification_lr(word2vec_cbow_emb,0.05)

Epoch: 1, loss: 0.69581, train_acc: 0.51
Epoch: 2, loss: 1.26033, train_acc: 0.49
Epoch: 3, loss: 0.71003, train_acc: 0.51
Epoch: 4, loss: 0.70892, train_acc: 0.51
Epoch: 5, loss: 0.69570, train_acc: 0.51
Epoch: 6, loss: 0.69749, train_acc: 0.49
Epoch: 7, loss: 0.69933, train_acc: 0.49
Epoch: 8, loss: 0.69715, train_acc: 0.49
Epoch: 9, loss: 0.69377, train_acc: 0.50
Epoch: 10, loss: 0.69233, train_acc: 0.50
Epoch: 11, loss: 0.69352, train_acc: 0.51
Epoch: 12, loss: 0.69505, train_acc: 0.51
Epoch: 13, loss: 0.69481, train_acc: 0.51
Epoch: 14, loss: 0.69342, train_acc: 0.51
Epoch: 15, loss: 0.69248, train_acc: 0.50
Epoch: 16, loss: 0.69260, train_acc: 0.51
Epoch: 17, loss: 0.69318, train_acc: 0.52
Epoch: 18, loss: 0.69332, train_acc: 0.51
Epoch: 19, loss: 0.69270, train_acc: 0.51
Epoch: 20, loss: 0.69176, train_acc: 0.51
Epoch: 21, loss: 0.69115, train_acc: 0.50
Epoch: 22, loss: 0.69114, train_acc: 0.52
Epoch: 23, loss: 0.69142, train_acc: 0.51
Epoch: 24, loss: 0.69150, train_acc: 0.51
E

In [ ]:
wv_cbow_model = Word2Vec(sentences = text_final, size=100, window=5, min_count=5, workers=2, sg=0)

In [ ]:
wv_sg_model = Word2Vec(sentences = text_final, size=100, window=5, min_count=5, workers=2, sg=1)

In [ ]:
def original_embedding(word_model):

  emb_table = []
  for i, word in enumerate(word_list):
    if word in word_model:
      emb_table.append(word_model[word])
    else:
      emb_table.append([0]*emb_dim)
  emb_table = np.array(emb_table)

  return emb_table

In [ ]:
word2vec_cbow_emb = original_embedding(wv_cbow_model.wv)

In [ ]:
word2vec_cbow_emb = update_embedding0(word2vec_cbow_emb)

weight adjusted tensor(1.9868, dtype=torch.float64, grad_fn=<DivBackward0>)
Epoch: 1, loss: 0.69307, train_acc: 0.52
weight adjusted tensor(1.9856, dtype=torch.float64, grad_fn=<DivBackward0>)
Epoch: 2, loss: 0.69385, train_acc: 0.50
weight adjusted tensor(1.9849, dtype=torch.float64, grad_fn=<DivBackward0>)
Epoch: 3, loss: 0.69284, train_acc: 0.50
weight adjusted tensor(1.9860, dtype=torch.float64, grad_fn=<DivBackward0>)
Epoch: 4, loss: 0.69248, train_acc: 0.50
weight adjusted tensor(1.9879, dtype=torch.float64, grad_fn=<DivBackward0>)
Epoch: 5, loss: 0.69106, train_acc: 0.53
weight adjusted tensor(1.9899, dtype=torch.float64, grad_fn=<DivBackward0>)
Epoch: 6, loss: 0.69074, train_acc: 0.54
weight adjusted tensor(1.9917, dtype=torch.float64, grad_fn=<DivBackward0>)
Epoch: 7, loss: 0.69023, train_acc: 0.54
weight adjusted tensor(1.9935, dtype=torch.float64, grad_fn=<DivBackward0>)
Epoch: 8, loss: 0.68887, train_acc: 0.55
weight adjusted tensor(1.9950, dtype=torch.float64, grad_fn=<Div

In [ ]:
original_word2vec_cbow_acc = classification(np.array(list(word2vec_cbow_emb.values())))

Epoch: 1, loss: 0.69318, train_acc: 0.51
Epoch: 2, loss: 0.68986, train_acc: 0.51
Epoch: 3, loss: 0.68492, train_acc: 0.51
Epoch: 4, loss: 0.68015, train_acc: 0.67
Epoch: 5, loss: 0.67538, train_acc: 0.67
Epoch: 6, loss: 0.66891, train_acc: 0.70
Epoch: 7, loss: 0.66064, train_acc: 0.67
Epoch: 8, loss: 0.65133, train_acc: 0.67
Epoch: 9, loss: 0.63696, train_acc: 0.71
Epoch: 10, loss: 0.61976, train_acc: 0.74
Epoch: 11, loss: 0.59155, train_acc: 0.77
Epoch: 12, loss: 0.56170, train_acc: 0.82
Epoch: 13, loss: 0.74749, train_acc: 0.51
Epoch: 14, loss: 0.57733, train_acc: 0.70
Epoch: 15, loss: 0.59213, train_acc: 0.72
Epoch: 16, loss: 0.63318, train_acc: 0.61
Epoch: 17, loss: 0.62343, train_acc: 0.64
Epoch: 18, loss: 0.60859, train_acc: 0.71
Epoch: 19, loss: 0.59995, train_acc: 0.79
Epoch: 20, loss: 0.59629, train_acc: 0.84
Epoch: 21, loss: 0.59531, train_acc: 0.80
Epoch: 22, loss: 0.59518, train_acc: 0.75
Epoch: 23, loss: 0.59462, train_acc: 0.73
Epoch: 24, loss: 0.59274, train_acc: 0.72
E

In [ ]:
new_word2vec_cbow_emb = update_embedding(word2vec_cbow_emb)

weight adjusted tensor(1.9857, dtype=torch.float64, grad_fn=<DivBackward0>)
Epoch: 1, loss: -1.29183, train_acc: 0.50
weight adjusted tensor(1.9860, dtype=torch.float64, grad_fn=<DivBackward0>)
Epoch: 2, loss: -1.28818, train_acc: 0.50
weight adjusted tensor(1.9812, dtype=torch.float64, grad_fn=<DivBackward0>)
Epoch: 3, loss: -1.28828, train_acc: 0.50
weight adjusted tensor(1.9760, dtype=torch.float64, grad_fn=<DivBackward0>)
Epoch: 4, loss: -1.28368, train_acc: 0.50
weight adjusted tensor(1.9732, dtype=torch.float64, grad_fn=<DivBackward0>)
Epoch: 5, loss: -1.27994, train_acc: 0.50
weight adjusted tensor(1.9713, dtype=torch.float64, grad_fn=<DivBackward0>)
Epoch: 6, loss: -1.27871, train_acc: 0.50
weight adjusted tensor(1.9699, dtype=torch.float64, grad_fn=<DivBackward0>)
Epoch: 7, loss: -1.27886, train_acc: 0.50
weight adjusted tensor(1.9690, dtype=torch.float64, grad_fn=<DivBackward0>)
Epoch: 8, loss: -1.27910, train_acc: 0.57
weight adjusted tensor(1.9637, dtype=torch.float64, grad

In [ ]:
word2vec_cbow_acc = classification(np.array(list(new_word2vec_cbow_emb.values())))

Epoch: 1, loss: 0.69345, train_acc: 0.51
Epoch: 2, loss: 0.68922, train_acc: 0.55
Epoch: 3, loss: 0.68438, train_acc: 0.71
Epoch: 4, loss: 0.67936, train_acc: 0.67
Epoch: 5, loss: 0.67387, train_acc: 0.64
Epoch: 6, loss: 0.66668, train_acc: 0.70
Epoch: 7, loss: 0.65743, train_acc: 0.74
Epoch: 8, loss: 0.64579, train_acc: 0.75
Epoch: 9, loss: 0.62851, train_acc: 0.77
Epoch: 10, loss: 0.60453, train_acc: 0.76
Epoch: 11, loss: 0.65521, train_acc: 0.64
Epoch: 12, loss: 0.64789, train_acc: 0.51
Epoch: 13, loss: 0.60752, train_acc: 0.56
Epoch: 14, loss: 0.59313, train_acc: 0.72
Epoch: 15, loss: 0.59891, train_acc: 0.79
Epoch: 16, loss: 0.60398, train_acc: 0.80
Epoch: 17, loss: 0.60053, train_acc: 0.82
Epoch: 18, loss: 0.59302, train_acc: 0.87
Epoch: 19, loss: 0.58662, train_acc: 0.91
Epoch: 20, loss: 0.58329, train_acc: 0.87
Epoch: 21, loss: 0.58227, train_acc: 0.83
Epoch: 22, loss: 0.57972, train_acc: 0.82
Epoch: 23, loss: 0.57178, train_acc: 0.82
Epoch: 24, loss: 0.55977, train_acc: 0.85
E

### Skip-Gram

In [ ]:
word2vec_sg_emb = original_embedding(wv_sg_model.wv)

In [ ]:
original_word2vec_sg_acc = classification_lr(word2vec_sg_emb,0.05)

Epoch: 1, loss: 0.69360, train_acc: 0.49
Epoch: 2, loss: 1.42767, train_acc: 0.51
Epoch: 3, loss: 0.83891, train_acc: 0.49
Epoch: 4, loss: 0.70168, train_acc: 0.50
Epoch: 5, loss: 0.69249, train_acc: 0.51
Epoch: 6, loss: 0.69527, train_acc: 0.51
Epoch: 7, loss: 0.69660, train_acc: 0.51
Epoch: 8, loss: 0.69555, train_acc: 0.51
Epoch: 9, loss: 0.69385, train_acc: 0.51
Epoch: 10, loss: 0.69272, train_acc: 0.51
Epoch: 11, loss: 0.69234, train_acc: 0.52
Epoch: 12, loss: 0.69245, train_acc: 0.52
Epoch: 13, loss: 0.69271, train_acc: 0.51
Epoch: 14, loss: 0.69288, train_acc: 0.49
Epoch: 15, loss: 0.69286, train_acc: 0.49
Epoch: 16, loss: 0.69260, train_acc: 0.51
Epoch: 17, loss: 0.69215, train_acc: 0.51
Epoch: 18, loss: 0.69158, train_acc: 0.52
Epoch: 19, loss: 0.69101, train_acc: 0.54
Epoch: 20, loss: 0.69051, train_acc: 0.52
Epoch: 21, loss: 0.69017, train_acc: 0.52
Epoch: 22, loss: 0.68997, train_acc: 0.53
Epoch: 23, loss: 0.68982, train_acc: 0.53
Epoch: 24, loss: 0.68957, train_acc: 0.53
E

In [ ]:
def original_embedding(word_model):

  emb_table = []
  for i, word in enumerate(word_list):
    if word in word_model:
      emb_table.append(word_model[word])
    else:
      emb_table.append([0]*emb_dim)
  emb_table = np.array(emb_table)

  return emb_table

In [ ]:
word2vec_sg_emb = original_embedding(wv_sg_model.wv)

In [ ]:
word2vec_sg_emb = update_embedding0(word2vec_sg_emb)

weight adjusted tensor(4.0285, dtype=torch.float64, grad_fn=<DivBackward0>)
Epoch: 1, loss: 0.69382, train_acc: 0.50
weight adjusted tensor(4.0575, dtype=torch.float64, grad_fn=<DivBackward0>)
Epoch: 2, loss: 0.69310, train_acc: 0.50
weight adjusted tensor(4.0281, dtype=torch.float64, grad_fn=<DivBackward0>)
Epoch: 3, loss: 0.69229, train_acc: 0.50
weight adjusted tensor(4.1044, dtype=torch.float64, grad_fn=<DivBackward0>)
Epoch: 4, loss: 0.69105, train_acc: 0.53
weight adjusted tensor(4.1818, dtype=torch.float64, grad_fn=<DivBackward0>)
Epoch: 5, loss: 0.69008, train_acc: 0.58
weight adjusted tensor(4.2585, dtype=torch.float64, grad_fn=<DivBackward0>)
Epoch: 6, loss: 0.68896, train_acc: 0.56
weight adjusted tensor(4.2581, dtype=torch.float64, grad_fn=<DivBackward0>)
Epoch: 7, loss: 0.68716, train_acc: 0.59
weight adjusted tensor(4.2600, dtype=torch.float64, grad_fn=<DivBackward0>)
Epoch: 8, loss: 0.68458, train_acc: 0.62
weight adjusted tensor(4.2574, dtype=torch.float64, grad_fn=<Div

In [ ]:
original_word2vec_sg_acc = classification(np.array(list(word2vec_sg_emb.values())))

Epoch: 1, loss: 1.38512, train_acc: 0.49
Epoch: 2, loss: 1.35037, train_acc: 0.51
Epoch: 3, loss: 1.33071, train_acc: 0.50
Epoch: 4, loss: 1.24682, train_acc: 0.80
Epoch: 5, loss: 1.19085, train_acc: 0.53
Epoch: 6, loss: 1.13679, train_acc: 0.73
Epoch: 7, loss: 1.17588, train_acc: 0.64
Epoch: 8, loss: 0.95349, train_acc: 0.87
Epoch: 9, loss: 8.49511, train_acc: 0.51
Epoch: 10, loss: 6.54510, train_acc: 0.51
Epoch: 11, loss: 2.97803, train_acc: 0.59
Epoch: 12, loss: 3.61270, train_acc: 0.49
Epoch: 13, loss: 2.73977, train_acc: 0.49
Epoch: 14, loss: 1.61658, train_acc: 0.49
Epoch: 15, loss: 1.37915, train_acc: 0.59
Epoch: 16, loss: 1.67767, train_acc: 0.51
Epoch: 17, loss: 1.73736, train_acc: 0.51
Epoch: 18, loss: 1.59877, train_acc: 0.51
Epoch: 19, loss: 1.42715, train_acc: 0.51
Epoch: 20, loss: 1.34381, train_acc: 0.61
Epoch: 21, loss: 1.37249, train_acc: 0.49
Epoch: 22, loss: 1.44160, train_acc: 0.49
Epoch: 23, loss: 1.47604, train_acc: 0.49
Epoch: 24, loss: 1.45728, train_acc: 0.49
E

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
new_word2vec_sg_emb = update_embedding(word2vec_sg_emb)

AttributeError: ignored

In [ ]:
word2vec_sg_acc = classification(np.array(list(new_word2vec_sg_emb.values())))

### Skip-Gram Fast Text

In [ ]:
from gensim.models import FastText
fasttext_sg_model = FastText(sentences = text_final, size=100, window=5, min_count=5, workers=2, sg=1)

In [ ]:
fasttext_sg_emb = original_embedding(fasttext_sg_model.wv)

In [ ]:
original_fasttext_sg_acc = classification(fasttext_sg_emb)

Epoch: 1, loss: 1.38620, train_acc: 0.51
Epoch: 2, loss: 1.53363, train_acc: 0.49
Epoch: 3, loss: 1.44042, train_acc: 0.52
Epoch: 4, loss: 1.39534, train_acc: 0.51
Epoch: 5, loss: 1.38359, train_acc: 0.52
Epoch: 6, loss: 1.38704, train_acc: 0.49
Epoch: 7, loss: 1.38968, train_acc: 0.49
Epoch: 8, loss: 1.38738, train_acc: 0.49
Epoch: 9, loss: 1.38502, train_acc: 0.53
Epoch: 10, loss: 1.38418, train_acc: 0.51
Epoch: 11, loss: 1.38442, train_acc: 0.51
Epoch: 12, loss: 1.38481, train_acc: 0.51
Epoch: 13, loss: 1.38465, train_acc: 0.51
Epoch: 14, loss: 1.38393, train_acc: 0.51
Epoch: 15, loss: 1.38310, train_acc: 0.53
Epoch: 16, loss: 1.38255, train_acc: 0.53
Epoch: 17, loss: 1.38233, train_acc: 0.52
Epoch: 18, loss: 1.38224, train_acc: 0.51
Epoch: 19, loss: 1.38194, train_acc: 0.52
Epoch: 20, loss: 1.38117, train_acc: 0.51
Epoch: 21, loss: 1.37992, train_acc: 0.51
Epoch: 22, loss: 1.37847, train_acc: 0.53
Epoch: 23, loss: 1.37715, train_acc: 0.55
Epoch: 24, loss: 1.37539, train_acc: 0.54
E

In [ ]:
new_fasttext_sg_emb0 = update_embedding0(fasttext_sg_emb)

weight adjusted tensor(55.8022, dtype=torch.float64, grad_fn=<DivBackward0>)
Epoch: 1, loss: -55.10660, train_acc: 0.50
weight adjusted tensor(55.7918, dtype=torch.float64, grad_fn=<DivBackward0>)
Epoch: 2, loss: -55.09784, train_acc: 0.50
weight adjusted tensor(55.7890, dtype=torch.float64, grad_fn=<DivBackward0>)
Epoch: 3, loss: -55.09553, train_acc: 0.50
weight adjusted tensor(55.7872, dtype=torch.float64, grad_fn=<DivBackward0>)
Epoch: 4, loss: -55.09550, train_acc: 0.50
weight adjusted tensor(55.7853, dtype=torch.float64, grad_fn=<DivBackward0>)
Epoch: 5, loss: -55.09433, train_acc: 0.53
weight adjusted tensor(55.7823, dtype=torch.float64, grad_fn=<DivBackward0>)
Epoch: 6, loss: -55.09168, train_acc: 0.50
weight adjusted tensor(55.7778, dtype=torch.float64, grad_fn=<DivBackward0>)
Epoch: 7, loss: -55.08825, train_acc: 0.50
weight adjusted tensor(55.7728, dtype=torch.float64, grad_fn=<DivBackward0>)
Epoch: 8, loss: -55.08494, train_acc: 0.55
weight adjusted tensor(55.7685, dtype=to

In [ ]:
new_fasttext_sg_acc0 = classification(np.array(list(new_fasttext_sg_emb0.values())))

Epoch: 1, loss: 1.38620, train_acc: 0.49
Epoch: 2, loss: 1.49030, train_acc: 0.51
Epoch: 3, loss: 1.35728, train_acc: 0.53
Epoch: 4, loss: 1.37308, train_acc: 0.49
Epoch: 5, loss: 1.34995, train_acc: 0.52
Epoch: 6, loss: 1.33696, train_acc: 0.65
Epoch: 7, loss: 1.32161, train_acc: 0.59
Epoch: 8, loss: 1.25858, train_acc: 0.78
Epoch: 9, loss: 1.81885, train_acc: 0.49
Epoch: 10, loss: 1.23752, train_acc: 0.66
Epoch: 11, loss: 1.32774, train_acc: 0.52
Epoch: 12, loss: 1.30053, train_acc: 0.52
Epoch: 13, loss: 1.27695, train_acc: 0.54
Epoch: 14, loss: 1.26081, train_acc: 0.71
Epoch: 15, loss: 1.25090, train_acc: 0.78
Epoch: 16, loss: 1.23298, train_acc: 0.75
Epoch: 17, loss: 1.14135, train_acc: 0.82
Epoch: 18, loss: 1.65368, train_acc: 0.52
Epoch: 19, loss: 1.10419, train_acc: 0.79
Epoch: 20, loss: 1.20906, train_acc: 0.61
Epoch: 21, loss: 1.16502, train_acc: 0.66
Epoch: 22, loss: 1.11113, train_acc: 0.81
Epoch: 23, loss: 1.10780, train_acc: 0.80
Epoch: 24, loss: 1.07289, train_acc: 0.79
E

In [ ]:
new_fasttext_sg_emb1 = update_embedding1(fasttext_sg_emb)

sparsemans loss -1.90059405040544
Epoch: 1, loss: -1.20734, train_acc: 0.49
sparsemans loss -1.8952811334824597
Epoch: 2, loss: -1.20225, train_acc: 0.50
sparsemans loss -1.8928501279838972
Epoch: 3, loss: -1.20110, train_acc: 0.50
sparsemans loss -1.8966098999593415
Epoch: 4, loss: -1.20543, train_acc: 0.50
sparsemans loss -1.88631788288062
Epoch: 5, loss: -1.19687, train_acc: 0.58
sparsemans loss -1.8779271574022895
Epoch: 6, loss: -1.18972, train_acc: 0.55
sparsemans loss -1.8555805621186023
Epoch: 7, loss: -1.16914, train_acc: 0.55
sparsemans loss -1.8461393406930384
Epoch: 8, loss: -1.16288, train_acc: 0.62
sparsemans loss -1.8321833784016888
Epoch: 9, loss: -1.15285, train_acc: 0.65
sparsemans loss -1.8169716394141864
Epoch: 10, loss: -1.14315, train_acc: 0.65
sparsemans loss -1.8034650112109287
Epoch: 11, loss: -1.13946, train_acc: 0.68
sparsemans loss -1.7980024253246756
Epoch: 12, loss: -1.14790, train_acc: 0.65
sparsemans loss -1.781077753006392
Epoch: 13, loss: -1.16267, tra

In [ ]:
new_fasttext_sg_acc1 = classification(np.array(list(new_fasttext_sg_emb1.values())))

Epoch: 1, loss: 1.38438, train_acc: 0.51
Epoch: 2, loss: 1.33613, train_acc: 0.63
Epoch: 3, loss: 1.93855, train_acc: 0.51
Epoch: 4, loss: 1.29098, train_acc: 0.69
Epoch: 5, loss: 1.35251, train_acc: 0.49
Epoch: 6, loss: 1.33449, train_acc: 0.49
Epoch: 7, loss: 1.30623, train_acc: 0.71
Epoch: 8, loss: 1.30315, train_acc: 0.61
Epoch: 9, loss: 1.25998, train_acc: 0.67
Epoch: 10, loss: 2.52516, train_acc: 0.49
Epoch: 11, loss: 1.29200, train_acc: 0.58
Epoch: 12, loss: 1.38105, train_acc: 0.51
Epoch: 13, loss: 1.37025, train_acc: 0.51
Epoch: 14, loss: 1.33637, train_acc: 0.51
Epoch: 15, loss: 1.31234, train_acc: 0.53
Epoch: 16, loss: 1.30179, train_acc: 0.72
Epoch: 17, loss: 1.30205, train_acc: 0.78
Epoch: 18, loss: 1.30777, train_acc: 0.59
Epoch: 19, loss: 1.30914, train_acc: 0.53
Epoch: 20, loss: 1.29731, train_acc: 0.55
Epoch: 21, loss: 1.27330, train_acc: 0.67
Epoch: 22, loss: 1.24735, train_acc: 0.79
Epoch: 23, loss: 1.22678, train_acc: 0.79
Epoch: 24, loss: 1.20489, train_acc: 0.74
E

In [ ]:
new_fasttext_sg_emb2 = update_embedding2(fasttext_sg_emb)

weight adjusted tensor(0.5580, dtype=torch.float64, grad_fn=<DivBackward0>)
Epoch: 1, loss: 0.13534, train_acc: 0.50
weight adjusted tensor(0.5581, dtype=torch.float64, grad_fn=<DivBackward0>)
Epoch: 2, loss: 0.13772, train_acc: 0.50
weight adjusted tensor(0.5581, dtype=torch.float64, grad_fn=<DivBackward0>)
Epoch: 3, loss: 0.13384, train_acc: 0.50
weight adjusted tensor(0.5581, dtype=torch.float64, grad_fn=<DivBackward0>)
Epoch: 4, loss: 0.13361, train_acc: 0.50
weight adjusted tensor(0.5581, dtype=torch.float64, grad_fn=<DivBackward0>)
Epoch: 5, loss: 0.13363, train_acc: 0.50
weight adjusted tensor(0.5581, dtype=torch.float64, grad_fn=<DivBackward0>)
Epoch: 6, loss: 0.13209, train_acc: 0.50
weight adjusted tensor(0.5581, dtype=torch.float64, grad_fn=<DivBackward0>)
Epoch: 7, loss: 0.13017, train_acc: 0.57
weight adjusted tensor(0.5581, dtype=torch.float64, grad_fn=<DivBackward0>)
Epoch: 8, loss: 0.12864, train_acc: 0.61
weight adjusted tensor(0.5581, dtype=torch.float64, grad_fn=<Div

In [ ]:
new_fasttext_sg_acc2 = classification(np.array(list(new_fasttext_sg_emb2.values())))

Epoch: 1, loss: 1.38657, train_acc: 0.51
Epoch: 2, loss: 1.40211, train_acc: 0.51
Epoch: 3, loss: 1.38923, train_acc: 0.49
Epoch: 4, loss: 1.34779, train_acc: 0.53
Epoch: 5, loss: 1.33421, train_acc: 0.64
Epoch: 6, loss: 1.32466, train_acc: 0.53
Epoch: 7, loss: 1.28893, train_acc: 0.61
Epoch: 8, loss: 1.21894, train_acc: 0.82
Epoch: 9, loss: 2.12332, train_acc: 0.66
Epoch: 10, loss: 1.47974, train_acc: 0.49
Epoch: 11, loss: 1.51658, train_acc: 0.49
Epoch: 12, loss: 1.43221, train_acc: 0.49
Epoch: 13, loss: 1.35848, train_acc: 0.49
Epoch: 14, loss: 1.30741, train_acc: 0.59
Epoch: 15, loss: 1.27761, train_acc: 0.60
Epoch: 16, loss: 1.37186, train_acc: 0.61
Epoch: 17, loss: 1.27791, train_acc: 0.67
Epoch: 18, loss: 1.27946, train_acc: 0.70
Epoch: 19, loss: 1.27999, train_acc: 0.74
Epoch: 20, loss: 1.27629, train_acc: 0.73
Epoch: 21, loss: 1.26692, train_acc: 0.71
Epoch: 22, loss: 1.24897, train_acc: 0.72
Epoch: 23, loss: 1.21653, train_acc: 0.73
Epoch: 24, loss: 1.16512, train_acc: 0.73
E

In [ ]:
new_fasttext_sg_emb3 = update_embedding3(fasttext_sg_emb)

sparsemans loss -1.90059405040544
word analogy weight adjusted tensor(55.8022, dtype=torch.float64, grad_fn=<DivBackward0>)
Epoch: 1, loss: -57.00965, train_acc: 0.50
sparsemans loss -1.8944081991591306
word analogy weight adjusted tensor(55.7997, dtype=torch.float64, grad_fn=<DivBackward0>)
Epoch: 2, loss: -56.99898, train_acc: 0.50
sparsemans loss -1.8997450767564492
word analogy weight adjusted tensor(55.7991, dtype=torch.float64, grad_fn=<DivBackward0>)
Epoch: 3, loss: -57.00714, train_acc: 0.55
sparsemans loss -1.8961988953868019
word analogy weight adjusted tensor(55.7972, dtype=torch.float64, grad_fn=<DivBackward0>)
Epoch: 4, loss: -57.00134, train_acc: 0.50
sparsemans loss -1.8716914840457342
word analogy weight adjusted tensor(55.7917, dtype=torch.float64, grad_fn=<DivBackward0>)
Epoch: 5, loss: -56.97184, train_acc: 0.50
sparsemans loss -1.8579242822092195
word analogy weight adjusted tensor(55.7850, dtype=torch.float64, grad_fn=<DivBackward0>)
Epoch: 6, loss: -56.95325, trai

In [ ]:
new_fasttext_sg_acc3 = classification(np.array(list(new_fasttext_sg_emb3.values())))

Epoch: 1, loss: 1.38675, train_acc: 0.51
Epoch: 2, loss: 1.34147, train_acc: 0.55
Epoch: 3, loss: 1.42465, train_acc: 0.51
Epoch: 4, loss: 1.38234, train_acc: 0.49
Epoch: 5, loss: 1.32938, train_acc: 0.50
Epoch: 6, loss: 1.31060, train_acc: 0.74
Epoch: 7, loss: 1.30738, train_acc: 0.65
Epoch: 8, loss: 1.30593, train_acc: 0.55
Epoch: 9, loss: 1.28323, train_acc: 0.59
Epoch: 10, loss: 1.23720, train_acc: 0.72
Epoch: 11, loss: 1.18486, train_acc: 0.81
Epoch: 12, loss: 1.00065, train_acc: 0.81
Epoch: 13, loss: 4.53466, train_acc: 0.51
Epoch: 14, loss: 1.31599, train_acc: 0.67
Epoch: 15, loss: 2.06187, train_acc: 0.49
Epoch: 16, loss: 1.74033, train_acc: 0.49
Epoch: 17, loss: 1.48807, train_acc: 0.49
Epoch: 18, loss: 1.35790, train_acc: 0.49
Epoch: 19, loss: 1.29848, train_acc: 0.56
Epoch: 20, loss: 1.27908, train_acc: 0.72
Epoch: 21, loss: 1.28404, train_acc: 0.71
Epoch: 22, loss: 1.30122, train_acc: 0.58
Epoch: 23, loss: 1.31249, train_acc: 0.54
Epoch: 24, loss: 1.30621, train_acc: 0.53
E

### CBOW FastText

In [ ]:
from gensim.models import FastText
fasttext_cbow_model = FastText(sentences = text_final, size=100, window=5, min_count=5, workers=2, sg=0)

In [ ]:
fasttext_cbow_emb = original_embedding(fasttext_cbow_model.wv)

In [ ]:
original_fasttext_cbow_acc = classification(fasttext_cbow_emb)

Epoch: 1, loss: 1.38772, train_acc: 0.49
Epoch: 2, loss: 2.55299, train_acc: 0.51
Epoch: 3, loss: 1.39202, train_acc: 0.51
Epoch: 4, loss: 1.44592, train_acc: 0.49
Epoch: 5, loss: 1.42329, train_acc: 0.49
Epoch: 6, loss: 1.38875, train_acc: 0.49
Epoch: 7, loss: 1.39399, train_acc: 0.51
Epoch: 8, loss: 1.40435, train_acc: 0.51
Epoch: 9, loss: 1.39679, train_acc: 0.51
Epoch: 10, loss: 1.38883, train_acc: 0.51
Epoch: 11, loss: 1.38559, train_acc: 0.51
Epoch: 12, loss: 1.38547, train_acc: 0.51
Epoch: 13, loss: 1.38637, train_acc: 0.50
Epoch: 14, loss: 1.38693, train_acc: 0.49
Epoch: 15, loss: 1.38670, train_acc: 0.50
Epoch: 16, loss: 1.38600, train_acc: 0.51
Epoch: 17, loss: 1.38537, train_acc: 0.53
Epoch: 18, loss: 1.38515, train_acc: 0.51
Epoch: 19, loss: 1.38535, train_acc: 0.51
Epoch: 20, loss: 1.38573, train_acc: 0.51
Epoch: 21, loss: 1.38603, train_acc: 0.51
Epoch: 22, loss: 1.38605, train_acc: 0.51
Epoch: 23, loss: 1.38578, train_acc: 0.52
Epoch: 24, loss: 1.38533, train_acc: 0.51
E

In [ ]:
new_fasttext_cbow_emb0 = update_embedding0(fasttext_cbow_emb)

weight adjusted tensor(124.7636, dtype=torch.float64, grad_fn=<DivBackward0>)
Epoch: 1, loss: 0.69407, train_acc: 0.50
weight adjusted tensor(124.7623, dtype=torch.float64, grad_fn=<DivBackward0>)
Epoch: 2, loss: 0.70304, train_acc: 0.50
weight adjusted tensor(124.7688, dtype=torch.float64, grad_fn=<DivBackward0>)
Epoch: 3, loss: 0.69359, train_acc: 0.50
weight adjusted tensor(124.7778, dtype=torch.float64, grad_fn=<DivBackward0>)
Epoch: 4, loss: 0.69323, train_acc: 0.50
weight adjusted tensor(124.7836, dtype=torch.float64, grad_fn=<DivBackward0>)
Epoch: 5, loss: 0.69546, train_acc: 0.50
weight adjusted tensor(124.7858, dtype=torch.float64, grad_fn=<DivBackward0>)
Epoch: 6, loss: 0.69447, train_acc: 0.50
weight adjusted tensor(124.7875, dtype=torch.float64, grad_fn=<DivBackward0>)
Epoch: 7, loss: 0.69230, train_acc: 0.50
weight adjusted tensor(124.7907, dtype=torch.float64, grad_fn=<DivBackward0>)
Epoch: 8, loss: 0.69093, train_acc: 0.50
weight adjusted tensor(124.7962, dtype=torch.flo

In [ ]:
new_fasttext_cbow_acc0 = classification(np.array(list(new_fasttext_cbow_emb0.values())))

Epoch: 1, loss: 1.38994, train_acc: 0.49
Epoch: 2, loss: 2.01311, train_acc: 0.51
Epoch: 3, loss: 1.37400, train_acc: 0.52
Epoch: 4, loss: 1.42297, train_acc: 0.49
Epoch: 5, loss: 1.40071, train_acc: 0.49
Epoch: 6, loss: 1.37598, train_acc: 0.49
Epoch: 7, loss: 1.37738, train_acc: 0.51
Epoch: 8, loss: 1.38681, train_acc: 0.51
Epoch: 9, loss: 1.38209, train_acc: 0.51
Epoch: 10, loss: 1.37298, train_acc: 0.51
Epoch: 11, loss: 1.36702, train_acc: 0.53
Epoch: 12, loss: 1.36459, train_acc: 0.61
Epoch: 13, loss: 1.36374, train_acc: 0.60
Epoch: 14, loss: 1.36165, train_acc: 0.50
Epoch: 15, loss: 1.35580, train_acc: 0.54
Epoch: 16, loss: 1.34585, train_acc: 0.69
Epoch: 17, loss: 1.33381, train_acc: 0.84
Epoch: 18, loss: 1.32035, train_acc: 0.68
Epoch: 19, loss: 1.29598, train_acc: 0.72
Epoch: 20, loss: 1.25746, train_acc: 0.89
Epoch: 21, loss: 1.18889, train_acc: 0.85
Epoch: 22, loss: 1.60519, train_acc: 0.51
Epoch: 23, loss: 1.26513, train_acc: 0.51
Epoch: 24, loss: 1.18100, train_acc: 0.80
E

In [ ]:
new_fasttext_cbow_emb1 = update_embedding1(fasttext_cbow_emb)

sparsemans loss -1.4949569395015492
Epoch: 1, loss: -0.80100, train_acc: 0.50
sparsemans loss -1.4907698149875188
Epoch: 2, loss: -0.78804, train_acc: 0.50
sparsemans loss -1.4861462470842848
Epoch: 3, loss: -0.79261, train_acc: 0.50
sparsemans loss -1.4880540833879483
Epoch: 4, loss: -0.79489, train_acc: 0.50
sparsemans loss -1.4490231950814858
Epoch: 5, loss: -0.75381, train_acc: 0.50
sparsemans loss -1.485635697257125
Epoch: 6, loss: -0.79143, train_acc: 0.50
sparsemans loss -1.4847552243676272
Epoch: 7, loss: -0.79260, train_acc: 0.50
sparsemans loss -1.4826713391432313
Epoch: 8, loss: -0.79173, train_acc: 0.81
sparsemans loss -1.444080815823441
Epoch: 9, loss: -0.75328, train_acc: 0.50
sparsemans loss -1.4751792297387172
Epoch: 10, loss: -0.78422, train_acc: 0.50
sparsemans loss -1.4762070128746156
Epoch: 11, loss: -0.78565, train_acc: 0.50
sparsemans loss -1.481781320445207
Epoch: 12, loss: -0.79229, train_acc: 0.50
sparsemans loss -1.4732064706001924
Epoch: 13, loss: -0.78511, t

In [ ]:
new_fasttext_cbow_acc1 = classification(np.array(list(new_fasttext_cbow_emb1.values())))

Epoch: 1, loss: 1.38869, train_acc: 0.49
Epoch: 2, loss: 1.88953, train_acc: 0.51
Epoch: 3, loss: 1.36667, train_acc: 0.58
Epoch: 4, loss: 1.41716, train_acc: 0.49
Epoch: 5, loss: 1.38889, train_acc: 0.49
Epoch: 6, loss: 1.37123, train_acc: 0.64
Epoch: 7, loss: 1.38609, train_acc: 0.51
Epoch: 8, loss: 1.38024, train_acc: 0.51
Epoch: 9, loss: 1.36812, train_acc: 0.51
Epoch: 10, loss: 1.36242, train_acc: 0.70
Epoch: 11, loss: 1.36141, train_acc: 0.60
Epoch: 12, loss: 1.36030, train_acc: 0.49
Epoch: 13, loss: 1.35444, train_acc: 0.50
Epoch: 14, loss: 1.34304, train_acc: 0.65
Epoch: 15, loss: 1.32853, train_acc: 0.78
Epoch: 16, loss: 1.30994, train_acc: 0.74
Epoch: 17, loss: 1.27182, train_acc: 0.84
Epoch: 18, loss: 1.22553, train_acc: 0.66
Epoch: 19, loss: 1.38885, train_acc: 0.51
Epoch: 20, loss: 1.15134, train_acc: 0.74
Epoch: 21, loss: 1.20928, train_acc: 0.54
Epoch: 22, loss: 1.15948, train_acc: 0.83
Epoch: 23, loss: 1.21512, train_acc: 0.55
Epoch: 24, loss: 1.17446, train_acc: 0.64
E

In [ ]:
new_fasttext_cbow_emb2 = update_embedding2(fasttext_cbow_emb)

weight adjusted tensor(1.2476, dtype=torch.float64, grad_fn=<DivBackward0>)
Epoch: 1, loss: -0.55369, train_acc: 0.50
weight adjusted tensor(1.2476, dtype=torch.float64, grad_fn=<DivBackward0>)
Epoch: 2, loss: -0.54366, train_acc: 0.50
weight adjusted tensor(1.2477, dtype=torch.float64, grad_fn=<DivBackward0>)
Epoch: 3, loss: -0.55412, train_acc: 0.50
weight adjusted tensor(1.2479, dtype=torch.float64, grad_fn=<DivBackward0>)
Epoch: 4, loss: -0.55449, train_acc: 0.50
weight adjusted tensor(1.2480, dtype=torch.float64, grad_fn=<DivBackward0>)
Epoch: 5, loss: -0.55224, train_acc: 0.50
weight adjusted tensor(1.2480, dtype=torch.float64, grad_fn=<DivBackward0>)
Epoch: 6, loss: -0.55318, train_acc: 0.50
weight adjusted tensor(1.2481, dtype=torch.float64, grad_fn=<DivBackward0>)
Epoch: 7, loss: -0.55542, train_acc: 0.50
weight adjusted tensor(1.2481, dtype=torch.float64, grad_fn=<DivBackward0>)
Epoch: 8, loss: -0.55700, train_acc: 0.50
weight adjusted tensor(1.2482, dtype=torch.float64, grad

In [ ]:
new_fasttext_cbow_acc2 = classification(np.array(list(new_fasttext_cbow_emb2.values())))

Epoch: 1, loss: 1.38693, train_acc: 0.51
Epoch: 2, loss: 1.35701, train_acc: 0.51
Epoch: 3, loss: 1.42046, train_acc: 0.51
Epoch: 4, loss: 1.65681, train_acc: 0.49
Epoch: 5, loss: 1.34079, train_acc: 0.55
Epoch: 6, loss: 1.33993, train_acc: 0.51
Epoch: 7, loss: 1.36084, train_acc: 0.51
Epoch: 8, loss: 1.35029, train_acc: 0.51
Epoch: 9, loss: 1.33386, train_acc: 0.63
Epoch: 10, loss: 1.33244, train_acc: 0.61
Epoch: 11, loss: 1.33269, train_acc: 0.50
Epoch: 12, loss: 1.31053, train_acc: 0.63
Epoch: 13, loss: 1.28841, train_acc: 0.83
Epoch: 14, loss: 1.27756, train_acc: 0.53
Epoch: 15, loss: 1.21204, train_acc: 0.91
Epoch: 16, loss: 1.24637, train_acc: 0.53
Epoch: 17, loss: 1.21217, train_acc: 0.54
Epoch: 18, loss: 1.12852, train_acc: 0.74
Epoch: 19, loss: 1.09009, train_acc: 0.86
Epoch: 20, loss: 1.07568, train_acc: 0.77
Epoch: 21, loss: 0.97243, train_acc: 0.85
Epoch: 22, loss: 0.89983, train_acc: 0.81
Epoch: 23, loss: 1.02267, train_acc: 0.72
Epoch: 24, loss: 2.53454, train_acc: 0.51
E

In [ ]:
new_fasttext_cbow_emb3 = update_embedding3(fasttext_cobw_emb)

NameError: ignored

In [ ]:
new_fasttext_cbow_acc3 = classification(np.array(list(new_fasttext_cbow_emb3.values())))

### Glove Twitter

In [ ]:
import gensim.downloader as api
wv_emb_twitter_100 = api.load("glove-twitter-100")

[==================================================] 100.0% 387.1/387.1MB downloaded


In [ ]:
wv_twitter_100_emb = original_embedding(wv_emb_twitter_100)

In [ ]:
wv_twitter_100_emb = update_embedding4(wv_emb_twitter_100)

In [ ]:
original_wv_twitter_100_acc = classification(wv_twitter_100_emb)

Epoch: 1, loss: 0.69247, train_acc: 0.55
Epoch: 2, loss: 0.68491, train_acc: 0.57
Epoch: 3, loss: 0.67719, train_acc: 0.64
Epoch: 4, loss: 0.66882, train_acc: 0.64
Epoch: 5, loss: 0.65890, train_acc: 0.65
Epoch: 6, loss: 0.64652, train_acc: 0.65
Epoch: 7, loss: 0.63067, train_acc: 0.67
Epoch: 8, loss: 0.60907, train_acc: 0.68
Epoch: 9, loss: 0.57998, train_acc: 0.69
Epoch: 10, loss: 0.55097, train_acc: 0.72
Epoch: 11, loss: 0.69185, train_acc: 0.72
Epoch: 12, loss: 0.57014, train_acc: 0.69
Epoch: 13, loss: 0.57164, train_acc: 0.69
Epoch: 14, loss: 0.52904, train_acc: 0.73
Epoch: 15, loss: 0.52702, train_acc: 0.74
Epoch: 16, loss: 0.54105, train_acc: 0.73
Epoch: 17, loss: 0.54283, train_acc: 0.73
Epoch: 18, loss: 0.52438, train_acc: 0.76
Epoch: 19, loss: 0.50173, train_acc: 0.77
Epoch: 20, loss: 0.49858, train_acc: 0.77
Epoch: 21, loss: 0.49244, train_acc: 0.78
Epoch: 22, loss: 0.47887, train_acc: 0.79
Epoch: 23, loss: 0.48439, train_acc: 0.80
Epoch: 24, loss: 0.48269, train_acc: 0.79
E

In [ ]:
new_wv_twitter_100_emb = update_embedding(wv_twitter_100_emb)

weight adjusted tensor(178.3759, dtype=torch.float64, grad_fn=<DivBackward0>)
Epoch: 1, loss: -177.68254, train_acc: 0.49
weight adjusted tensor(178.3767, dtype=torch.float64, grad_fn=<DivBackward0>)
Epoch: 2, loss: -177.69086, train_acc: 0.59
weight adjusted tensor(178.4548, dtype=torch.float64, grad_fn=<DivBackward0>)
Epoch: 3, loss: -177.77700, train_acc: 0.68
weight adjusted tensor(178.5329, dtype=torch.float64, grad_fn=<DivBackward0>)
Epoch: 4, loss: -177.86382, train_acc: 0.68
weight adjusted tensor(178.4546, dtype=torch.float64, grad_fn=<DivBackward0>)
Epoch: 5, loss: -177.79628, train_acc: 0.70
weight adjusted tensor(178.5678, dtype=torch.float64, grad_fn=<DivBackward0>)
Epoch: 6, loss: -177.92357, train_acc: 0.71
weight adjusted tensor(178.4108, dtype=torch.float64, grad_fn=<DivBackward0>)
Epoch: 7, loss: -177.78543, train_acc: 0.72
weight adjusted tensor(178.4880, dtype=torch.float64, grad_fn=<DivBackward0>)
Epoch: 8, loss: -177.89009, train_acc: 0.73
weight adjusted tensor(1

In [ ]:
new_wv_twitter_100_acc = classification(np.array(list(new_wv_twitter_100_emb.values())))

Epoch: 1, loss: 0.69147, train_acc: 0.52
Epoch: 2, loss: 0.68041, train_acc: 0.65
Epoch: 3, loss: 0.66873, train_acc: 0.68
Epoch: 4, loss: 0.65552, train_acc: 0.72
Epoch: 5, loss: 0.63955, train_acc: 0.71
Epoch: 6, loss: 0.61873, train_acc: 0.72
Epoch: 7, loss: 0.59034, train_acc: 0.74
Epoch: 8, loss: 0.54948, train_acc: 0.75
Epoch: 9, loss: 0.48956, train_acc: 0.77
Epoch: 10, loss: 0.47611, train_acc: 0.78
Epoch: 11, loss: 0.72179, train_acc: 0.75
Epoch: 12, loss: 0.45080, train_acc: 0.79
Epoch: 13, loss: 0.50928, train_acc: 0.73
Epoch: 14, loss: 0.45852, train_acc: 0.78
Epoch: 15, loss: 0.42565, train_acc: 0.83
Epoch: 16, loss: 0.42779, train_acc: 0.82
Epoch: 17, loss: 0.44414, train_acc: 0.83
Epoch: 18, loss: 0.45745, train_acc: 0.82
Epoch: 19, loss: 0.45783, train_acc: 0.81
Epoch: 20, loss: 0.44559, train_acc: 0.82
Epoch: 21, loss: 0.42789, train_acc: 0.84
Epoch: 22, loss: 0.41112, train_acc: 0.85
Epoch: 23, loss: 0.39779, train_acc: 0.85
Epoch: 24, loss: 0.38639, train_acc: 0.85
E

#### Tricks: Normalization

In [ ]:
new_wv_twitter_100_emb = update_embedding2(wv_twitter_100_emb)

weight adjusted tensor(1.7838, dtype=torch.float64, grad_fn=<DivBackward0>)
Epoch: 1, loss: -1.09022, train_acc: 0.49
weight adjusted tensor(1.7838, dtype=torch.float64, grad_fn=<DivBackward0>)
Epoch: 2, loss: -1.09742, train_acc: 0.58
weight adjusted tensor(1.7845, dtype=torch.float64, grad_fn=<DivBackward0>)
Epoch: 3, loss: -1.10577, train_acc: 0.66
weight adjusted tensor(1.7861, dtype=torch.float64, grad_fn=<DivBackward0>)
Epoch: 4, loss: -1.11550, train_acc: 0.68
weight adjusted tensor(1.7869, dtype=torch.float64, grad_fn=<DivBackward0>)
Epoch: 5, loss: -1.12586, train_acc: 0.68
weight adjusted tensor(1.7877, dtype=torch.float64, grad_fn=<DivBackward0>)
Epoch: 6, loss: -1.13921, train_acc: 0.68
weight adjusted tensor(1.7861, dtype=torch.float64, grad_fn=<DivBackward0>)
Epoch: 7, loss: -1.15427, train_acc: 0.71
weight adjusted tensor(1.7869, dtype=torch.float64, grad_fn=<DivBackward0>)
Epoch: 8, loss: -1.17763, train_acc: 0.71
weight adjusted tensor(1.7876, dtype=torch.float64, grad

In [ ]:
new_wv_twitter_100_acc = classification(np.array(list(new_wv_twitter_100_emb.values())))

Epoch: 1, loss: 0.69275, train_acc: 0.52
Epoch: 2, loss: 0.68252, train_acc: 0.62
Epoch: 3, loss: 0.67214, train_acc: 0.68
Epoch: 4, loss: 0.66041, train_acc: 0.71
Epoch: 5, loss: 0.64615, train_acc: 0.73
Epoch: 6, loss: 0.62763, train_acc: 0.73
Epoch: 7, loss: 0.60222, train_acc: 0.75
Epoch: 8, loss: 0.56505, train_acc: 0.76
Epoch: 9, loss: 0.50527, train_acc: 0.78
Epoch: 10, loss: 0.41376, train_acc: 0.81
Epoch: 11, loss: 0.50615, train_acc: 0.79
Epoch: 12, loss: 0.52596, train_acc: 0.80
Epoch: 13, loss: 0.40977, train_acc: 0.83
Epoch: 14, loss: 0.42078, train_acc: 0.82
Epoch: 15, loss: 0.44753, train_acc: 0.80
Epoch: 16, loss: 0.45412, train_acc: 0.80
Epoch: 17, loss: 0.44617, train_acc: 0.82
Epoch: 18, loss: 0.43203, train_acc: 0.83
Epoch: 19, loss: 0.41658, train_acc: 0.85
Epoch: 20, loss: 0.39904, train_acc: 0.87
Epoch: 21, loss: 0.37248, train_acc: 0.87
Epoch: 22, loss: 0.33222, train_acc: 0.87
Epoch: 23, loss: 0.34239, train_acc: 0.86
Epoch: 24, loss: 0.31395, train_acc: 0.87
E

In [ ]:
new_wv_twitter_100_acc = classification(np.array(list(new_wv_twitter_100_emb.values())))

Epoch: 1, loss: 0.69241, train_acc: 0.51
Epoch: 2, loss: 0.68178, train_acc: 0.68
Epoch: 3, loss: 0.67093, train_acc: 0.71
Epoch: 4, loss: 0.65875, train_acc: 0.73
Epoch: 5, loss: 0.64441, train_acc: 0.72
Epoch: 6, loss: 0.62621, train_acc: 0.72
Epoch: 7, loss: 0.60185, train_acc: 0.74
Epoch: 8, loss: 0.56806, train_acc: 0.74
Epoch: 9, loss: 0.51742, train_acc: 0.77
Epoch: 10, loss: 0.43883, train_acc: 0.80
Epoch: 11, loss: 0.56838, train_acc: 0.77
Epoch: 12, loss: 0.77710, train_acc: 0.73
Epoch: 13, loss: 0.48537, train_acc: 0.81
Epoch: 14, loss: 0.39130, train_acc: 0.81
Epoch: 15, loss: 0.46752, train_acc: 0.76
Epoch: 16, loss: 0.46066, train_acc: 0.77
Epoch: 17, loss: 0.43546, train_acc: 0.81
Epoch: 18, loss: 0.42315, train_acc: 0.84
Epoch: 19, loss: 0.42288, train_acc: 0.85
Epoch: 20, loss: 0.42783, train_acc: 0.84
Epoch: 21, loss: 0.43227, train_acc: 0.85
Epoch: 22, loss: 0.43167, train_acc: 0.84
Epoch: 23, loss: 0.42225, train_acc: 0.85
Epoch: 24, loss: 0.40048, train_acc: 0.85
E

## New Embedding Loss: Spearmans Loss

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import math
import numpy
from operator import itemgetter
from numpy.linalg import norm
import sys
import os
import gzip

EPSILON = 1e-6

def euclidean(vec1, vec2):
  diff = vec1 - vec2
  return math.sqrt(diff.dot(diff))

def cosine_sim(vec1, vec2):
  vec1 += EPSILON * numpy.ones(len(vec1))
  vec2 += EPSILON * numpy.ones(len(vec1))
  return vec1.dot(vec2)/(norm(vec1)*norm(vec2)+1)

def assign_ranks(item_dict):
  ranked_dict = {}
  sorted_list = [(key, val) for (key, val) in sorted(item_dict.items(),
                                                     key=itemgetter(1),
                                                     reverse=True)]
  for i, (key, val) in enumerate(sorted_list):
    same_val_indices = []
    for j, (key2, val2) in enumerate(sorted_list):
      if val2 == val:
        same_val_indices.append(j+1)
    if len(same_val_indices) == 1:
      ranked_dict[key] = i+1
    else:
      ranked_dict[key] = 1.*sum(same_val_indices)/(len(same_val_indices)+1)
  return ranked_dict

def correlation(dict1, dict2):
  avg1 = 1.*sum([val for key, val in dict1.iteritems()])/len(dict1)
  avg2 = 1.*sum([val for key, val in dict2.iteritems()])/len(dict2)
  numr, den1, den2 = (0., 0., 0.)
  for val1, val2 in zip(dict1.itervalues(), dict2.itervalues()):
    numr += (val1 - avg1) * (val2 - avg2)
    den1 += (val1 - avg1) ** 2
    den2 += (val2 - avg2) ** 2
  return numr / math.sqrt(den1 * den2)

def spearmans_rho(ranked_dict1, ranked_dict2):
  assert len(ranked_dict1) == len(ranked_dict2)
  if len(ranked_dict1) == 0 or len(ranked_dict2) == 0:
    return 0.
  x_avg = 1.*sum([val for val in ranked_dict1.values()])/len(ranked_dict1)
  y_avg = 1.*sum([val for val in ranked_dict2.values()])/len(ranked_dict2)
  num, d_x, d_y = (0., 0., 0.)
  for key in ranked_dict1.keys():
    xi = ranked_dict1[key]
    yi = ranked_dict2[key]
    num += (xi-x_avg)*(yi-y_avg)
    d_x += (xi-x_avg)**2
    d_y += (yi-y_avg)**2
  return num/(math.sqrt(d_x*d_y)+1)

from collections import Counter
from operator import itemgetter

''' Read all the word vectors and normalize them '''
def read_word_vectors(filename):    
  word_vecs = {}
  #if filename.endswith('.gz'): file_object = gzip.open(filename, 'r')
  #else: file_object = open(filename, 'r')
  print(filename)
  file_object = open(str(filename),"r")

  for line_num, line in enumerate(file_object):
    line = line.strip().lower()
    word = line.split()[0]
    word_vecs[word] = numpy.zeros(len(line.split())-1, dtype=float)
    for index, vec_val in enumerate(line.split()[1:]):
      word_vecs[word][index] = float(vec_val)      
    ''' normalize weight vector '''
    word_vecs[word] /= math.sqrt((word_vecs[word]**2).sum() + 1e-6)        

  sys.stderr.write("Vectors read from: "+filename+" \n")
  return word_vecs

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
word_sim_dir = "/content/drive/MyDrive/en"
word_vec_file = "/content/drive/MyDrive/embed/filtered.txt"

def sim_loss_cal(word_sim_dir,word_vec_file):
  word_vecs = read_word_vectors(word_vec_file)
  #print(word_vecs)
  print('=================================================================================')
  print("%6s" %"Serial", "%20s" % "Dataset", "%15s" % "Num Pairs", "%15s" % "Not found", "%15s" % "Rho")
  print('=================================================================================')

  avg_spearmans_rho = 0

  for i, filename in enumerate(os.listdir(word_sim_dir)):
      manual_dict, auto_dict = ({}, {})
      not_found, total_size = (0, 0)
      for line in open(os.path.join(word_sim_dir, filename),'r'):
          line = line.strip().lower()
          word1, word2, val = line.split()
          if word1 in word_vecs and word2 in word_vecs:
              manual_dict[(word1, word2)] = float(val)
              auto_dict[(word1, word2)] = cosine_sim(word_vecs[word1], word_vecs[word2])
          else:
              not_found += 1
          total_size += 1
      print("%6s" % str(i+1), "%20s" % filename, "%15s" % str(total_size), end=""),
      print("%15s" % str(not_found), end=""),
      print("%15.4f" % spearmans_rho(assign_ranks(manual_dict), assign_ranks(auto_dict)))
      avg_spearmans_rho +=spearmans_rho(assign_ranks(manual_dict), assign_ranks(auto_dict))
  avg_spearmans_rho /= len(os.listdir(word_sim_dir))
  return avg_spearmans_rho

In [ ]:
sim_loss = sim_loss_cal(word_sim_dir,word_vec_file)

/content/drive/MyDrive/embed/filtered.txt


Vectors read from: /content/drive/MyDrive/embed/filtered.txt 


Serial              Dataset       Num Pairs       Not found             Rho
     1     EN-MTurk-287.txt             287              0         0.6580
     2         EN-MC-30.txt              30              0         0.7767
     3    EN-SIMLEX-999.txt             999              0         0.3386
     4     EN-MEN-TR-3k.txt            3000              0         0.7303
     5   EN-RW-STANFORD.txt            2034             96         0.4093
     6         EN-RG-65.txt              65              0         0.7715
     7     EN-MTurk-771.txt             771              0         0.6027
     8      EN-VERB-143.txt             144              0         0.3334
     9  EN-SimVerb-3500.txt            3500              2         0.2004
    10    EN-WS-353-REL.txt             252              0         0.6027
    11        EN-YP-130.txt             130              0         0.4274
    12    EN-WS-353-SIM.txt             203              0         0.7226
    13    EN-WS-353-ALL.txt         

In [ ]:
print(sim_loss)

0.5561924900080913


In [ ]:
wordvecs = read_word_vectors(word_vec_file)

/content/drive/MyDrive/embed/filtered.txt


Vectors read from: /content/drive/MyDrive/embed/filtered.txt 


In [ ]:
wordvecs["performances"].shape

(100,)

In [ ]:
word_sim_dir = "/content/drive/MyDrive/en"
word_vec_file = "/content/drive/MyDrive/embed/filtered.txt"

def sim_loss_cal2(word_sim_dir,word_vec_file):
  word_vecs = word_vec_file
  avg_spearmans_rho = 0

  for i, filename in enumerate(os.listdir(word_sim_dir)):
      manual_dict, auto_dict = ({}, {})
      not_found, total_size = (0, 0)
      for line in open(os.path.join(word_sim_dir, filename),'r'):
          line = line.strip().lower()
          word1, word2, val = line.split()
          if word1 in word_vecs and word2 in word_vecs:
              manual_dict[(word1, word2)] = float(val)
              auto_dict[(word1, word2)] = cosine_sim(word_vecs[word1], word_vecs[word2])
          else:
              not_found += 1
          total_size += 1
      avg_spearmans_rho +=spearmans_rho(assign_ranks(manual_dict), assign_ranks(auto_dict))
  avg_spearmans_rho /= len(os.listdir(word_sim_dir))
  return avg_spearmans_rho

In [ ]:
word2vec_cbow_emb = original_embedding(wv_cbow_model.wv)

In [ ]:
original_word2vec_cbow_acc = classification(word2vec_cbow_emb)

Epoch: 1, loss: 0.69314, train_acc: 0.51
Epoch: 2, loss: 0.69793, train_acc: 0.49
Epoch: 3, loss: 0.69274, train_acc: 0.52
Epoch: 4, loss: 0.69385, train_acc: 0.51
Epoch: 5, loss: 0.69473, train_acc: 0.51
Epoch: 6, loss: 0.69358, train_acc: 0.51
Epoch: 7, loss: 0.69251, train_acc: 0.51
Epoch: 8, loss: 0.69235, train_acc: 0.52
Epoch: 9, loss: 0.69276, train_acc: 0.51
Epoch: 10, loss: 0.69303, train_acc: 0.50
Epoch: 11, loss: 0.69286, train_acc: 0.51
Epoch: 12, loss: 0.69244, train_acc: 0.51
Epoch: 13, loss: 0.69205, train_acc: 0.51
Epoch: 14, loss: 0.69187, train_acc: 0.52
Epoch: 15, loss: 0.69192, train_acc: 0.51
Epoch: 16, loss: 0.69206, train_acc: 0.51
Epoch: 17, loss: 0.69209, train_acc: 0.51
Epoch: 18, loss: 0.69193, train_acc: 0.51
Epoch: 19, loss: 0.69169, train_acc: 0.50
Epoch: 20, loss: 0.69153, train_acc: 0.51
Epoch: 21, loss: 0.69150, train_acc: 0.52
Epoch: 22, loss: 0.69155, train_acc: 0.52
Epoch: 23, loss: 0.69156, train_acc: 0.51
Epoch: 24, loss: 0.69148, train_acc: 0.51
E

In [ ]:
new_word2vec_cbow_emb = update_embedding(word2vec_cbow_emb)

weight adjusted tensor(1.9857, dtype=torch.float64, grad_fn=<DivBackward0>)
Epoch: 1, loss: -1.29256, train_acc: 0.50
weight adjusted tensor(1.9847, dtype=torch.float64, grad_fn=<DivBackward0>)
Epoch: 2, loss: -1.28544, train_acc: 0.50
weight adjusted tensor(1.9799, dtype=torch.float64, grad_fn=<DivBackward0>)
Epoch: 3, loss: -1.28737, train_acc: 0.51
weight adjusted tensor(1.9766, dtype=torch.float64, grad_fn=<DivBackward0>)
Epoch: 4, loss: -1.28364, train_acc: 0.50
weight adjusted tensor(1.9751, dtype=torch.float64, grad_fn=<DivBackward0>)
Epoch: 5, loss: -1.28094, train_acc: 0.50
weight adjusted tensor(1.9742, dtype=torch.float64, grad_fn=<DivBackward0>)
Epoch: 6, loss: -1.28133, train_acc: 0.50
weight adjusted tensor(1.9739, dtype=torch.float64, grad_fn=<DivBackward0>)
Epoch: 7, loss: -1.28282, train_acc: 0.50
weight adjusted tensor(1.9740, dtype=torch.float64, grad_fn=<DivBackward0>)
Epoch: 8, loss: -1.28402, train_acc: 0.57
weight adjusted tensor(1.9698, dtype=torch.float64, grad

In [ ]:
word2vec_cbow_acc = classification(np.array(list(new_word2vec_cbow_emb.values())))

Epoch: 1, loss: 0.69490, train_acc: 0.51
Epoch: 2, loss: 0.69061, train_acc: 0.51
Epoch: 3, loss: 0.68603, train_acc: 0.60
Epoch: 4, loss: 0.68101, train_acc: 0.64
Epoch: 5, loss: 0.67631, train_acc: 0.67
Epoch: 6, loss: 0.67075, train_acc: 0.68
Epoch: 7, loss: 0.66329, train_acc: 0.69
Epoch: 8, loss: 0.65433, train_acc: 0.72
Epoch: 9, loss: 0.64329, train_acc: 0.72
Epoch: 10, loss: 0.62681, train_acc: 0.75
Epoch: 11, loss: 0.60556, train_acc: 0.78
Epoch: 12, loss: 0.61014, train_acc: 0.68
Epoch: 13, loss: 0.63011, train_acc: 0.52
Epoch: 14, loss: 0.60568, train_acc: 0.57
Epoch: 15, loss: 0.58615, train_acc: 0.75
Epoch: 16, loss: 0.59287, train_acc: 0.81
Epoch: 17, loss: 0.59152, train_acc: 0.80
Epoch: 18, loss: 0.57932, train_acc: 0.85
Epoch: 19, loss: 0.56710, train_acc: 0.90
Epoch: 20, loss: 0.56022, train_acc: 0.88
Epoch: 21, loss: 0.55776, train_acc: 0.85
Epoch: 22, loss: 0.55357, train_acc: 0.83
Epoch: 23, loss: 0.54263, train_acc: 0.83
Epoch: 24, loss: 0.52832, train_acc: 0.84
E

In [ ]:
word2vec_cbow_emb = original_embedding(wv_cbow_model.wv)

In [ ]:
new_word2vec_cbow_emb = update_embedding3(word2vec_cbow_emb)

sparsemans loss -0.9950910951643707
word analogy weight adjusted tensor(1.9857, dtype=torch.float64, grad_fn=<DivBackward0>)
Epoch: 1, loss: -2.28731, train_acc: 0.50
sparsemans loss -0.834683194312816
word analogy weight adjusted tensor(1.9855, dtype=torch.float64, grad_fn=<DivBackward0>)
Epoch: 2, loss: -2.12311, train_acc: 0.50
sparsemans loss -0.1977031225436035
word analogy weight adjusted tensor(1.9860, dtype=torch.float64, grad_fn=<DivBackward0>)
Epoch: 3, loss: -1.49092, train_acc: 0.50
sparsemans loss 0.7584770149218548
word analogy weight adjusted tensor(1.9854, dtype=torch.float64, grad_fn=<DivBackward0>)
Epoch: 4, loss: -0.53449, train_acc: 0.50
sparsemans loss 0.9708299249305037
word analogy weight adjusted tensor(1.9857, dtype=torch.float64, grad_fn=<DivBackward0>)
Epoch: 5, loss: -0.32171, train_acc: 0.50
sparsemans loss 1.1207870272312181
word analogy weight adjusted tensor(1.9857, dtype=torch.float64, grad_fn=<DivBackward0>)
Epoch: 6, loss: -0.17254, train_acc: 0.50
sp

In [ ]:
word2vec_cbow_acc = classification(np.array(list(new_word2vec_cbow_emb.values())))

Epoch: 1, loss: 0.69328, train_acc: 0.50
Epoch: 2, loss: 0.68788, train_acc: 0.51
Epoch: 3, loss: 0.68239, train_acc: 0.63
Epoch: 4, loss: 0.67625, train_acc: 0.74
Epoch: 5, loss: 0.66879, train_acc: 0.74
Epoch: 6, loss: 0.65930, train_acc: 0.74
Epoch: 7, loss: 0.64661, train_acc: 0.75
Epoch: 8, loss: 0.62828, train_acc: 0.81
Epoch: 9, loss: 0.59846, train_acc: 0.86
Epoch: 10, loss: 0.56888, train_acc: 0.69
Epoch: 11, loss: 0.97630, train_acc: 0.50
Epoch: 12, loss: 0.78509, train_acc: 0.50
Epoch: 13, loss: 0.64426, train_acc: 0.55
Epoch: 14, loss: 0.59510, train_acc: 0.79
Epoch: 15, loss: 0.60502, train_acc: 0.73
Epoch: 16, loss: 0.62248, train_acc: 0.62
Epoch: 17, loss: 0.62086, train_acc: 0.54
Epoch: 18, loss: 0.61914, train_acc: 0.54
Epoch: 19, loss: 0.61855, train_acc: 0.55
Epoch: 20, loss: 0.61840, train_acc: 0.56
Epoch: 21, loss: 0.61830, train_acc: 0.59
Epoch: 22, loss: 0.61809, train_acc: 0.63
Epoch: 23, loss: 0.61769, train_acc: 0.66
Epoch: 24, loss: 0.61708, train_acc: 0.69
E